# 02 DATA TRANSFORMATION

## Overview of the Jupyter Notebook  

The objective of the Jupyter Notebook is to perform data transformation, specifically invoking preliminary exploratory data analysis (EDA) to reduce the dimensionality of the data and narrow the focus of the analysis.  
<br>
The Backblaze first quarter of 2022 hard drive snapshot data is read from the Parquet file into a PySpark DataFrame. Through aggregation, the data is examined. The manufacturer is determined to be missing from some of the hard drive models. Through research of models, data is transformed to pair the models with manufacturers.  
<br>
S.M.A.R.T. (Self-Monitoring, Analysis and Reporting Technology) or SMART attributes provide health indicators and statistics for the hard drive (S.M.A.R.T., 2022). There are 255 SMART attributes ranging from 0-254 (S.M.A.R.T., 2022). The SMART attributes represent a substantial amount of features in the hard drive snapshots for the machine learning classification problem. Unfortunately, the SMART attributes are manufacturer specific and across manufacturers lack standardization, have different implementations, or in some cases only select attributes (S.M.A.R.T., 2022). For compatibility only one manufacturer is selected.  
<br>
After accessing the various manufacturers a significant class imbalance is identified. There is a heavy skew toward hard drive snapshots that represent `functional` (or `0`) compared to the hard drive snapshots that represent `non-functional` (or `1`). Further analysis determined that Seagate has the most hard drive snapshots and therefore the most samples for both the majority class (`functional`) and the minority class (`non-functional`). The focus of the project is resolved to Seagate. A deeper probe is conducted on the various Seagate models and reveals a mixture of hard drive disks (HDD) and solid state drives (SSD). Analysis of both hard drive types together presents challenges due to inconsistencies on reporting of the S.M.A.R.T. (S.M.A.R.T., 2022). After research, Seagate (n.d.) stated that not all BIOS by manufacturers interpret SSD S.M.A.R.T. attributes correctly. Moreover, there are HDD and SSD S.M.A.R.T. attributes that overlap (S.M.A.R.T., 2022). The SSD models were only recently introduced into the reporting and constitute a small amount of hard drive snapshots. Rather than introduce inconsistencies into the analysis, the SDD models are removed and the focus is further narrowed to HDD models. To prevent bias and improve class imbalance HDD models are removed with only one class, the majority class (`functional`), resulting in the concentration of 14 Seagate HDD models.  
<br>
Further efforts are made to reduce the data dimensionality by removing columns that have 40% or more missing values. In addition, rows are also removed where there are 40% or more (18 columns or more) that have missing values. Columns with only one distinct value are also removed because one distinct value offers little importance.  
<br>
Lastly, a strategy is employed to create a window of snapshots leading up to the failure. Predicting a failure at the time of failure offers little benefit toward predictive maintenance. A literature review revealed varying attempts at creating a deteriorating window (i.e. a window leading up to failure). Choices for selecting a deteriortating window were based on intuition, uncorrectable errors, and performance from a neural network. Based on the prior research, a decision was made to use a 7 day deteriorating window. The 7 day deteriorating window increases the hard drive snapshots for the minority class (`non-functional`). For each hard drive that failed, the 6 days of hard drive snapshots leading up to the failure were re-classified from the majority class (`functional`) to the minority class (`non-functional`). The hard drive snapshot representing the day of failure remains classified as the minority class (`non-functional`). The results are combined into a dataframe and saved to a Parquet file.  

## Import modules and libraries  

In [1]:
# References
# https://stackoverflow.com/questions/65398794/what-does-this-mean-warningrootpyarrow-ignore-timezone-environment-variabl
# https://stackoverflow.com/questions/58987008/persisting-a-data-frame-in-pyspark2-does-not-work-when-a-storage-level-is-specif

import os

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import re
import shutil
from pathlib import Path

import pyarrow as pa
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Window as W
from pyspark.sql import functions as F

In [2]:
# Java is a requirement for PySpark. 
# Check Java installation and version.
!java -version

openjdk version "17.0.3-internal" 2022-04-19
OpenJDK Runtime Environment (build 17.0.3-internal+0-adhoc..src)
OpenJDK 64-Bit Server VM (build 17.0.3-internal+0-adhoc..src, mixed mode, sharing)


## Create tmp directory

The `tmp` directory is required for scracth space in PySpark. For instance, PySpark DataFrames are persisted to the scratch space. By default, the scratch space is saved to the `tmp` in the root directory (Apache Spark, n.d.). However, for Saturn Cloud, the `tmp` in the root directory is outside of the user space. As an alternative, a `tmp` directory needs to be created and used within the user space.  

In [3]:
# https://stackoverflow.com/questions/30057323/where-does-spark-actually-persist-rdds-on-disk
# https://stackoverflow.com/questions/69368590/pyspark-set-local-dir-to-avoid-java-io-ioexception-no-space-left-on-device
# https://stackoverflow.com/questions/40372655/how-to-set-spark-local-dir-property-from-spark-shell

# By default, PySpark persists PySpark DataFrames to '/tmp' directory, 
# which is outside the user space in Saturn Cloud.
# To overcome this issue, PySpark config 'spark.local.dir' can be 
# pointed to a specified directory, in this case, 'tmp' directory in 
# the user space.

# The current working directory is assigned to the 'cwd_path' variable.
cwd_path = Path.cwd()

# Try to make the 'tmp' directory, but if the directory already exists 
# then raise and print and excpetion.
try:
    os.makedirs(cwd_path.joinpath('tmp'), exist_ok=False)
except FileExistsError:
    print("Directory already exists")

## PySpark configuration  

PySpark is configured based on the working enviroment (i.e. CPU, RAM, Jupyter Notebook, etc.). To maximize performance, PySpark is intended to be used in a cluster confirguration. However, PySpark also provides a 'local' option for development and testing.  

In [4]:
# References
# https://spark.apache.org/docs/latest/configuration.html
# https://spark.apache.org/docs/latest/submitting-applications.html#master-urls
# https://stackoverflow.com/questions/30057323/where-does-spark-actually-persist-rdds-on-disk
# https://stackoverflow.com/questions/69368590/pyspark-set-local-dir-to-avoid-java-io-ioexception-no-space-left-on-device
# https://stackoverflow.com/questions/40372655/how-to-set-spark-local-dir-property-from-spark-shell

# Create a SparkSession with the PySpark configuration properties. 
# Driver memory is set to 75% of total memory (96g).
# The master URL is set to 'local' and 75% of logical CPUs (12).
# The Spark Local Dir is set to `tmp` directory within the user space.
# Support for Apache Arrow is enabled and if an error occurs fallback 
# is disabled and self-destruct enabled for data transfers.
# The display of the progress bar is enabled.
# Eager evaluation of Jupyter notebooks is enabled and allows for a 
# maximum rows of 100 and a maximum of 50 characters to be displayed.
spark = SparkSession.builder \
    .appName('Practicum_MSDS692') \
    .config('spark.driver.memory', '96g') \
    .config('spark.local.dir', '/home/jovyan/workspace/tmp') \
    .master('local[12]') \
    .config('spark.memory.storageFraction', '0.6') \
    .config('spark.sql.execution.arrow.pyspark.enabled', True) \
    .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False) \
    .config('spark.sql.execution.arrow.pyspark.selfDestruct.enabled', True) \
    .config('spark.ui.showConsoleProgress', True) \
    .config('spark.sql.repl.eagerEval.enabled', True) \
    .config('spark.sql.repl.eagerEval.maxNumRows', 100) \
    .config('spark.sql.repl.eagerEval.truncate', 50) \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/22 02:24:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/22 02:24:09 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [5]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkContext.setLogLevel.html

# As mentioned above, the default log level is set to 'WARN'. The 
# default log level is overridden to "OFF" level, which suppresses the
# warning messages.
spark.sparkContext.setLogLevel('OFF')

In [6]:
# References
# https://sparkbyexamples.com/spark/spark-get-the-current-sparkcontext-settings/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.sparkContext.html
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.SparkConf.html

# Verify the the PySpark SparkSession SparkContext properties.
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.driver.host',
  'w-elyon-msds692-fee39812bd1f4728bef3977b2e7f821e-6f8bc85dd55f5z'),
 ('spark.sql.repl.eagerEval.enabled', 'True'),
 ('spark.driver.memory', '96g'),
 ('spark.app.name', 'Practicum_MSDS692'),
 ('spark.executor.

## Notebook functions

PySpark does not have a 'shape' attribute similar to Pandas, where the number of rows and columns of a dataframe are returned. To overcome this limitation, a custom function is created to replicate the Pandas 'shape' attribue behavior.

In [7]:
# Reference:
# https://sparkbyexamples.com/pyspark/pyspark-dataframe-shape/
# https://stackoverflow.com/questions/39652767/how-to-find-the-size-or-shape-of-a-dataframe-in-pyspark

# A custom function ('sdf_shape') is created to combine Pyspark 
# DataFrame 'count()' function which returns the number of rows and
# the length ('len') of the Pyspark DataFrame 'columns' property which
# returns number of columns.
def sdf_shape(sdf):
    """
    Accepts a PySpark DataFrame and returns the number of rows and 
    columns.
    
    Keyword arguments:
    sdf -- the PySpark DataFrame
    """
    return (sdf.count(), len(sdf.columns))

## Read Parquet file into a dataframe  

The hard drive snapshot data for the first quarter of 2022 was previously read from comma separated value (CSV) files and written to a Parquet file. A dataframe is created by reading the Parquet file.  

In [8]:
# The current working directory is assigned to the 'cwd_path' variable.
cwd_path = Path.cwd()

# The 'data/parquet' directory is joined to the 'cwd_path' and assigned 
# to the 'data_path' variable.
pq_path = cwd_path.joinpath('data/parquet')

In [9]:
# Reference:
# https://spark.apache.org/docs/latest/sql-data-sources-parquet.html
# https://sparkbyexamples.com/pyspark/pyspark-read-and-write-parquet-file/
# https://sparkbyexamples.com/spark/spark-read-write-dataframe-parquet-example/
# https://stackoverflow.com/questions/48422137/efficient-way-to-read-specific-columns-from-parquet-file-in-spark

# A Parquet file is read into PySpark and assigned to PySpark 
# DataFrame.
q1_2022_sdf = spark.read.parquet(str(pq_path) + '/q1_2022-0.parquet')

## About the data

Important to highlight, unless specified the rows or observations do not represent individual hard drives. Instead a row or observation reflects a snapshot of a specific hard drive. For instance, a hard drive with a serial number of `S3001HBH` might have 90 snapshots for the first quarter of 2022 (a snapshot for every single day - 90 days). However, if the hard drive with the serial number of `ZTN0A6FX` only operated for 1 day, there would be 1 snapshot.  
<br>
The data consists of the following columns (Beach, 2015):
* Date
* Serial Number
* Model
* Capacity (Bytes)
* Failure
    * Functional - 0
    * Non-functional - 1
* S.M.A.R.T. Attributes
<br>
The S.M.A.R.T. attributes provide health indicators and statistics of the hard drive through monitoring (S.M.A.R.T., 2022). The S.M.A.R.T. attributes range from 0-254 and each attribute reports various conditions of the hard drive reliability (S.M.A.R.T., 2022). The raw values are determined by the manufacturer but can equate to counts. Normalized values range from 1 to 253, where 1 is the worst value, 100 is the default value, and 253 is the best value (S.M.A.R.T., 2022). The target or variable attempting to be predicted is the `failure` field. The failure field entails a 0 for functional and 1 for non-functional, making the project a machine learning binary classification problem.  
<br>
As previously stated, there is no standardization of S.M.A.R.T. attributes between manufacturers and there are inconsistencies between hard disk drives (HDD) and solid state drives (SSD) within manufacturers.  

## Verify type, shape, schema, and the first five records of each of the quarter of the dataframes  

Once a dataframe is created from the first quarter of 2022 data in the Parquet file, the dataframe is inspected. The dataframe is confirmed to be a `pyspark.sql.dataframe.DataFrame` type. Next, the shape of the dataframe is examined. The row and column account appear to match the metadata from the Parquet file. Lastly, the first 5 records are reviewed to identify any obvious issues. Initially, there appears to be 'null' values that will need dealt with.  

The PySpark DataFrame is confirmed to be a `pyspark.sql.dataframe.DataFrame` type.

In [10]:
# Confirm the returned data type is a PySpark DataFrame 
# ('pyspark.sql.dataframe.DataFrame').
type(q1_2022_sdf)

pyspark.sql.dataframe.DataFrame

The shape of PySpark DataFrame is 18,845,260 rows and 179 columns.

In [11]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_sdf)

(18845260, 179)

The schema of the dataframe appears to be consistent with the Parquet file schema displayed in the previous notebook.  

In [12]:
# Display the schema of the PySpark DataFrame.
# Confirm that the field names and types are correct.
q1_2022_sdf.printSchema()

root
 |-- date: date (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: long (nullable = true)
 |-- failure: long (nullable = true)
 |-- smart_1_normalized: long (nullable = true)
 |-- smart_1_raw: long (nullable = true)
 |-- smart_2_normalized: long (nullable = true)
 |-- smart_2_raw: long (nullable = true)
 |-- smart_3_normalized: long (nullable = true)
 |-- smart_3_raw: long (nullable = true)
 |-- smart_4_normalized: long (nullable = true)
 |-- smart_4_raw: long (nullable = true)
 |-- smart_5_normalized: long (nullable = true)
 |-- smart_5_raw: long (nullable = true)
 |-- smart_7_normalized: long (nullable = true)
 |-- smart_7_raw: long (nullable = true)
 |-- smart_8_normalized: long (nullable = true)
 |-- smart_8_raw: long (nullable = true)
 |-- smart_9_normalized: long (nullable = true)
 |-- smart_9_raw: long (nullable = true)
 |-- smart_10_normalized: long (nullable = true)
 |-- smart_10_raw: long (nullable = t

The first 5 records are reviewed for consistency and to identify any obvious issues. There are several columns that appear to have 'null' values that will need to be addressed.  

In [13]:
# The first 5 rows of the PySpark DataFrame ('q1_2021_sdf') are
# displayed.
q1_2022_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,null,null,90,0,100,12,100,0,87,495846641,null,null,89,9937,100,0,null,null,100,12,null,null,null,null,null,null,null,null,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,0,100,0,null,null,66,34,null,null,100,2,99,2641,34,34,null,null,null,null,100,0,100,0,200,0,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,9430,100,75539400520,100,139325292722,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,null,null,99,0,100,1,1

## Inspect the dataframe  

Grouping, counting, and sorting the dataframe by `model` reveals the quantity of hard drive snapshots per model. Some models contain manufacturer names such as DELLBOSS, HGST, HP, Hitachi, Seagate, TOSHIBA, and WDC. However, other models such as MTFDDAV240TCB, ST10000NM001G, etc. are misssing manufacturer names.  

In [14]:
# The PySpark DataFrame ('q1_2022_sdf') is grouped by 'model', a count 
# peformed by 'model', and sorted by 'model' in ascending order.
(
    q1_2022_sdf
    .groupBy('model')
    .count()
    .sort('model')
)

model,count
CT250MX500SSD1,11575
DELLBOSS VD,24383
HGST HDS5C4040ALE630,2340
HGST HDS724040ALE640,90
HGST HMS5C4040ALE640,312911
HGST HMS5C4040BLE640,1145719
HGST HMS5C4040BLE641,90
HGST HUH721010ALE600,1800
HGST HUH721212ALE600,234063
HGST HUH721212ALE604,1181921


Grouping, counting, and sorting the dataframe by `model` and using a distinct count on `serial_number` reveals the quantity of unique hard drives per model. The contrast between the quantity of hard drive snapshots and unique hard drives illustrates the amount of snapshots per unique hard drive. For example, the `CT250MX500SSD1` model has 11,575 hard drive snapshots (see above), but has 174 unique hard drives (see below).   

In [15]:
# References
# https://stackoverflow.com/questions/46421677/how-to-count-unique-id-after-groupby-in-pyspark
# https://stackoverflow.com/questions/36064777/count-the-distinct-elements-of-each-group-by-other-field-on-a-spark-1-6-datafram
# https://stackoverflow.com/questions/29988287/renaming-columns-for-pyspark-dataframe-aggregates

# The PySpark DataFrame ('q1_2022_sdf') is grouped by 'model', a count
# distinct aggregation on 'serial_name' is performed and aliased as 
# 'unique_serial_numbers', then sorted by 'model' in ascending order.
(
    q1_2022_sdf
    .groupBy('model')
    .agg(F.countDistinct('serial_number')
         .alias('unique_serial_numbers'))
    .sort('model')
)

model,unique_serial_numbers
CT250MX500SSD1,174
DELLBOSS VD,290
HGST HDS5C4040ALE630,26
HGST HDS724040ALE640,1
HGST HMS5C4040ALE640,3552
HGST HMS5C4040BLE640,12743
HGST HMS5C4040BLE641,1
HGST HUH721010ALE600,20
HGST HUH721212ALE600,2602
HGST HUH721212ALE604,13154


Aggregating the data by distinct count on `serial_number` reveals that there is a total of 212,404 quantity of unique hard drives for the first quarter of 2022.  

In [16]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.count_distinct.html

# The PySpark DataFrame ('q1_2022_sdf') a count distinct aggregation on 
# 'serial_name' is performed and aliased as 'unique_serial_numbers.
(
    q1_2022_sdf
    .agg(F.count_distinct('serial_number')
         .alias('unique_serial_numbers'))
)

unique_serial_numbers
212404


## Add the manufacturer to the model to the 2021 dataFrame

Through an exhaustive internet search on model, the manufacturer names were obtained and paired with each model with a missing manufacturer name. Models starting with `CT` were classified as `CRUCIAL`, `MT` as `MIRCRON`, `SS` as `DELL`, `HN` as `SEAGATE-SAMSUNG`, and `ST` as `SEAGATE`.

In [17]:
# References
# https://stackoverflow.com/questions/66458824/pyspark-when-column-value-starts-with-x-write-as-y
# https://amiradata.com/pyspark-concat-two-columns/
# https://stackoverflow.com/questions/54816878/multiple-when-condition-implementation-in-pyspark
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withColumn.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.when.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.when.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.startswith.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.contains.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.lit.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.otherwise.html

# The PySpark DataFrame ('q1_2022_sdf') 'model' column is replaced with 
# the existing name. The replaced 'model' column is updated using 
# a series of when conditions that checks if the model starts with 2 
# characters and if so, concantenates the manufacturer name to the
# model.
# If there is no match with the 2 characters, no update is made and the
# model remains the same. The results are saved to a PySpark DataFrame
# ('q1_2022_add_mfr_sdf').
q1_2022_add_mfr_sdf = (
    q1_2022_sdf
    .withColumn('model', 
                F.when(F.col('model').startswith('CT'), 
                       F.concat(F.lit('CRUCIAL '), 
                                F.col('model')))
                .when(F.col('model').startswith('MT'), 
                      F.concat(F.lit('MICRON '), 
                               F.col('model')))
                .when(F.col('model').startswith('SS'), 
                      F.concat(F.lit('DELL '), 
                               F.col('model')))
                .when(F.col('model').contains('HN'), 
                      F.concat(F.lit('SEAGATE-SAMSUNG '), 
                               F.col('model')))
                .when(F.col('model').startswith('ST'), 
                      F.concat(F.lit('SEAGATE '), 
                               F.col('model')))
                .otherwise(F.col('model')))
)

Previously, `ST14000NM001G` model was missing a manufacturer name. Now, the `ST14000NM001G` model is associated with the `SEAGATE` manufacturer. However, the manufacturer name is now concatenated with the `model`, which makes analysis by manufacturer challenging.  

In [18]:
# The first 5 rows of the PySpark DataFrame ('q1_2022_add_mfr_sdf') are
# displayed.
q1_2022_add_mfr_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
2022-01-01,ZLW18P9K,SEAGATE ST14000NM001G,14000519643136,0,73,20467240,null,null,90,0,100,12,100,0,87,495846641,null,null,89,9937,100,0,null,null,100,12,null,null,null,null,null,null,null,null,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,0,100,0,null,null,66,34,null,null,100,2,99,2641,34,34,null,null,null,null,100,0,100,0,200,0,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,9430,100,75539400520,100,139325292722,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-01-01,ZLW0EGC7,SEAGATE ST12000NM001G,12000138625024,0,84,228715872,null,n

## Split the model column into manufacturer and model columns  

The manufacturer name and model are split into two columns for analysis by manufacturer.

In [19]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-split-dataframe-column-into-multiple-columns/
# https://azurelib.com/withcolumn-usage-in-databricks-with-examples/
# https://stackoverflow.com/questions/55143035/pyspark-split-a-column-and-take-n-elements
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.upper.html
# https://stackoverflow.com/questions/15987650/regex-to-get-everything-after-the-first-space
# https://stackoverflow.com/questions/13750716/what-does-regular-expression-s-s-do
# https://stackoverflow.com/questions/27881366/regular-expressions-and
# https://stackoverflow.com/questions/46410887/pyspark-string-matching-to-create-new-column
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.trim.html
# https://sparkbyexamples.com/spark/spark-dataframe-cache-and-persist-explained/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.persist.html

# The PySpark DataFrame ('q1_2022_add_mfr_sdf') 'model' column is 
# replaced with the existing name. The replaced 'model' column casts 
# the model in uppercase.
# A new 'manufacturer' column is created by splitting the model on a 
# ' ' and getting the 0 item (1st word).
# The 'model' column is replaced again using regular expressions to 
# match any number of whitespace characters and captures all of the 
# items after the whitespace. 
# The 'trim' function is used to remove any surrounding whitespace from 
# the model.
# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is persisted to 
# optimize performance.
q1_2022_split_mfrmod_sdf = (
    q1_2022_add_mfr_sdf
    .withColumn('model', 
                F.upper(F.col('model')))
    .withColumn('manufacturer', 
                F.split(F.col('model'), ' ')
                .getItem(0))
    .withColumn('model', 
                F.regexp_extract(F.col('model'), 
                                 '(\s*) (.*)', 2))
    .withColumn('model', 
                F.trim(F.col('model')))
).persist()

Please note that the `model` column maintains position in the dataframe, but the `manufacturer` column is appended to the end of the dataframe.  

In [20]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_split_mfrmod_sdf') are displayed. The 'persist' is
# initiated, since 'limit' is an action.
q1_2022_split_mfrmod_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,null,null,90,0,100,12,100,0,87,495846641,null,null,89,9937,100,0,null,null,100,12,null,null,null,null,null,null,null,null,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,0,100,0,null,null,66,34,null,null,100,2,99,2641,34,34,null,null,null,null,100,0,100,0,200,0,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,9430,100,75539400520,100,139325292722,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,n

## Examination of manufacturer with the highest quantity of hard drive snapshots and unique hard drives

There are a total of 18,845,260 hard drive snapshots.  

In [21]:
# Display the shape (rows, columns) of the PySpark DataFrame 
# ('q1_2022_split_mfrmod_sdf') using the 'sdf_shape' function.
sdf_shape(q1_2022_split_mfrmod_sdf)

(18845260, 180)

Looking at hard drive snapshots by `manufacturer` for the first quarter of 2022, reveals that `SEAGATE` has the most equating to at 9,463,775.  

In [22]:
# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a count peformed by 'manufacturer', and sorted by 
# 'count' in descending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer')
    .count()
    .sort('count', ascending=False)
)

manufacturer,count
SEAGATE,9463775
TOSHIBA,4307266
HGST,3961278
WDC,1030806
SEAGATE-SAMSUNG,37537
DELLBOSS,24383
CRUCIAL,11575
MICRON,8100
HITACHI,540


Looking at unique hard drives by `manufacturer` for first quarter 2022, reveals that `SEAGATE` has the most equating to at 106,634.  

In [23]:
# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a count distinct aggregation on 'manufacturer' is
# performed, aliased as 'unique_serial_numbers, and sorted in
# in descending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer')
    .agg(F.count_distinct('serial_number')
         .alias('unique_serial_numbers'))
    .sort('unique_serial_numbers', ascending=False)
)

manufacturer,unique_serial_numbers
SEAGATE,106634
TOSHIBA,48111
HGST,44167
WDC,12509
SEAGATE-SAMSUNG,423
DELLBOSS,290
CRUCIAL,174
MICRON,90
HITACHI,6


## Examination of manufacturer with the highest quantity of hard drive snapshots and unique hard drives with failures (non-functional)

When split by target into `functional` and `non-functional` hard drive snaphsots, `SEAGATE` has hard drives with 9,463,338 `functional` and 437 `non-functional` snapshots, the most for both categories in the first quarter of 2022.  

In [24]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.withColumnRenamed.html

# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count is performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# Lastly, a sort is performed on the 'non-functional' column in
# descending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .sort('non-functional', ascending=False)
)

manufacturer,functional,non-functional
SEAGATE,9463338,437
TOSHIBA,4307129,137
HGST,3961201,77
SEAGATE-SAMSUNG,37526,11
WDC,1030804,2
DELLBOSS,24383,null
CRUCIAL,11575,null
MICRON,8100,null
HITACHI,540,null


When split by target into `functional` and `non-functional` unique hard drives, `SEAGATE` has hard drives with 106,625 `functional` and 437 `non-functional` unique hard drives, the most for both categories in the first quarter of 2022.  

In [25]:
# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', a count distinct 
# aggregation on 'serial_number' is performed, and aliased as 
# 'unique_serial_numbers'.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# Lastly, a sort is performed on the 'non-functional' column in
# descending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer')
    .pivot('failure')
    .agg(F.count_distinct('serial_number')
         .alias('unique_serial_numbers'))
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .sort('non-functional', ascending=False)
)

manufacturer,functional,non-functional
SEAGATE,106625,437
TOSHIBA,48108,137
HGST,44151,77
SEAGATE-SAMSUNG,423,11
WDC,12509,2
DELLBOSS,290,null
CRUCIAL,174,null
MICRON,90,null
HITACHI,6,null


Although `SEAGATE` has the highest quantity of hard drive failures (`non-functional`) based on snapshots, SEAGATE-SAMSUNG has the largest `failure_percent` (the percentage of drive failures) at 0.0293%. However, SEAGATE-SAMSUNG only has 11 hard drive failure (`non-functional`) snapshots. The amount of hard drive failure (`non-functional`) snapshots present challenges with a minimal amount of observations to train and test the machine learning model. `SEAGATE` provides the greatest amount of samples for hard drive failure (`non-functional`) snapshots for analysis equating to 437.  

The `failure_percent` also reveals that there is an extreme imbalance in the data. The majority class (`functional`) has a large proportion of observations compared to the minority class (`non-functional`). The imbalance issue is acknowledged and will be addressed later.  

In [26]:
# References
# https://stackoverflow.com/questions/56074407/pyspark-how-to-create-a-calculated-column-in-pyspark-sql

# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count is performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'non-functional' column in
# descending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
    .sort('failure_percent', ascending=False)
)

manufacturer,functional,non-functional,failure_percent
SEAGATE-SAMSUNG,37526,11,0.0293
SEAGATE,9463338,437,0.0046
TOSHIBA,4307129,137,0.0032
HGST,3961201,77,0.0019
WDC,1030804,2,2.0E-4
DELLBOSS,24383,null,null
CRUCIAL,11575,null,null
MICRON,8100,null,null
HITACHI,540,null,null


The same applies when looking at the highest quantity of failures based on unique hard drives. SEAGATE-SAMSUNG still has the largest `failure_percent` (the percentage of drive failures) at 2.5346%, but the minimal amount of observations still presents a challenge.  

In [27]:
# References
# https://stackoverflow.com/questions/56074407/pyspark-how-to-create-a-calculated-column-in-pyspark-sql

# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', a count distinct 
# aggregation on 'serial_number' is performed, and aliased as 
# 'unique_serial_numbers'.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'non-functional' column in
# descending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer')
    .pivot('failure')
    .agg(F.count_distinct('serial_number')
         .alias('unique_serial_numbers'))
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
    .sort('failure_percent', ascending=False)
)

manufacturer,functional,non-functional,failure_percent
SEAGATE-SAMSUNG,423,11,2.5346
SEAGATE,106625,437,0.4082
TOSHIBA,48108,137,0.284
HGST,44151,77,0.1741
WDC,12509,2,0.016
DELLBOSS,290,null,null
CRUCIAL,174,null,null
MICRON,90,null,null
HITACHI,6,null,null


## SEAGATE is selected as the manufacturer of focus for the project

Looking at hard drive snapshots for the first quarter of 2022, the manufacturers range from 2 to 437 `non-functional` snapshots. Although proper sample size for machine learning can be subjective with varying influential factors having more samples can benefit the machine learning process. `SEAGATE` has the most samples for both classes and will be the focus of the project.  

## Examination of SEAGATE hard disk drives (HDD) models and solid state drives (SSD) models

The `SEAGATE` models include both hard disk drives (HDD) and solid state drives (SSD). Of the hard drives, there are five SSDs models. Backblaze previously only reported on HDDs and did not start reporting on SSDs until 2022 (Klein, 2022). Seagate (n.d.) also states that some manufacturer's BIOS do not interpret S.M.A.R.T. SSD attributes correctly. Moreover, Wikipedia (2022) lists six S.M.A.R.T. attributes associated with SSDs, and in one instance an attribute (Attribute ID# 230) has a different attribute name for each hard drive type - HDD (GMR Head Amplitude) and SSD (Drive Life Protection Status). For consistency of the analysis and to prevent incompatible values being combined, only HDDs will be evaluated. Klein (2022) validated that the Seagate SSD models include the word `SSD` or start with `ZA`. Looking at the `SEAGATE` modeles, the first 5 models appear to be `SSD`.  

In [28]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-where-filter/
# https://sqlandhadoop.com/pyspark-filter-25-examples-to-teach-you-everything/
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html

# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer' and 'model', and a count performed.
# A filter is applied to only return the `SEAGATE` manufacturer.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer', 'model')
    .count()
    .filter(F.col('manufacturer') == 'SEAGATE')
    .sort('model')
)

manufacturer,model,count
SEAGATE,BARRACUDA 120 SSD ZA250CM10003,98091
SEAGATE,BARRACUDA SSD ZA2000CM10002,270
SEAGATE,BARRACUDA SSD ZA250CM10002,50533
SEAGATE,BARRACUDA SSD ZA500CM10002,1620
SEAGATE,SSD,9587
SEAGATE,ST10000NM001G,1028
SEAGATE,ST10000NM0086,107151
SEAGATE,ST12000NM0007,118349
SEAGATE,ST12000NM0008,1815311
SEAGATE,ST12000NM001G,1099549


The `SEAGATE` `SSD` models can be filtered down to 5 models and only account for 6 hard drive failure (`non-functional`) snapshots.  

In [29]:
# References
# https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.Column.like.html

# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count performed.
# A filter is applied to only return the `SEAGATE` manufacturer.
# An additional filter is applied to match 'SSD' in the model.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer', 'model')
    .pivot('failure')
    .count()
    .filter(F.col('manufacturer') == 'SEAGATE')
    .filter(F.col('model')
            .like('%SSD%'))
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
    .sort('model')
)

manufacturer,model,functional,non-functional,failure_percent
SEAGATE,BARRACUDA 120 SSD ZA250CM10003,98088,3,0.0031
SEAGATE,BARRACUDA SSD ZA2000CM10002,270,null,null
SEAGATE,BARRACUDA SSD ZA250CM10002,50531,2,0.004
SEAGATE,BARRACUDA SSD ZA500CM10002,1620,null,null
SEAGATE,SSD,9586,1,0.0104


The `SEAGATE` `SSD` models equate to a total of 160,101 hard drive snapshots - 160,095 `functional` and 6 `non-functional`.  

In [30]:
# The PySpark DataFrame ('q1_2022_split_mfrmod_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count performed.
# A filter is applied to only return the `SEAGATE` manufacturer.
# An additional filter is applied to match 'SSD' in the model.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sum aggregation is performed on 'functional' and
# 'non-functional' and aliased.
(
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer', 'model')
    .pivot('failure')
    .count()
    .filter(F.col('manufacturer') == 'SEAGATE')
    .filter(F.col('model')
            .like('%SSD%'))
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
).agg(
    F.sum('functional')
    .alias('functional_sum'), 
    F.sum('non-functional')
    .alias('non-functional_sum')
)

functional_sum,non-functional_sum
160095,6


## Removing SEAGATE SSD models to avoid S.M.A.R.T. attributes inconsistencies

Removing the `SEAGATE` `SSD` model snapshots will eliminate the S.M.A.R.T attributes inconsistencies between the `HDD` models and the `SSD` models. In addition, the removal of the `SEAGATE` `SSD` model hard drive snapshots will also contribute toward improving the class imbalance by removing 160,095 hard drive snapshots from the majority class (`functional`) and only 6 hard drive snapshots from the minority class (`non-functional`).  

In [31]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-where-filter/
# https://sqlandhadoop.com/pyspark-filter-25-examples-to-teach-you-everything/
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.filter.html
# https://spark.apache.org/docs/latest/api/python//reference/pyspark.sql/api/pyspark.sql.Column.like.html

# A PySpark Dataframe ('q1_2022_seagate_hdd_sdf') is created using
# the PySpark DataFrame ('q1_2022_split_mfrmod_sdf'), which is grouped
# 'manufacturer', a pivot on 'failure', and a count is performed.
# A filter is applied to only return the `SEAGATE` manufacturer.
# An additional filter is applied to *not* match 'SSD' in the model.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# Lastly, a sort is performed on the 'non-functional' column in
# ascending order.
q1_2022_seagate_hdd_sdf = (
    q1_2022_split_mfrmod_sdf
    .groupBy('manufacturer', 'model')
    .pivot('failure')
    .count()
    .filter(F.col('manufacturer') == 'SEAGATE')
    .filter(~F.col('model')
            .like('%SSD%'))
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .sort('model')
)

After removing the `SEAGATE` `SSD` models, there is still an extreme imbalance in the data at 0.0046%.  

In [32]:
# The PySpark DataFrame ('q1_2022_seagate_hdd_sdf') is grouped by 
# 'manufacturer' and a sum is performed.
# The 'sum(functional)' column is renamed to 'functional' and the 
# 'sum(non-functional)' column is renamed to 'non-functional'.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'non-functional' column in
# ascending order.
(
    q1_2022_seagate_hdd_sdf
    .groupBy('manufacturer')
    .sum()
    .withColumnRenamed('sum(functional)', 
                       'functional')
    .withColumnRenamed('sum(non-functional)', 
                       'non-functional')
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
)

manufacturer,functional,non-functional,failure_percent
SEAGATE,9303243,431,0.0046


## Examination of SEAGATE HDD models with no (null) hard drive failures

With the `SEAGATE` `SSD` model snapshots removed, there are still several hard drive models that have null values for hard drive failure (`non-functional`) snapshots. 

The models with only the majority class (`functional`) for the target can present issues for the machine learning model because the model might be biased and simply default to selecting the majority class (`functional`) for these hard drive model.  

In [33]:
q1_2022_seagate_hdd_sdf

manufacturer,model,functional,non-functional
SEAGATE,ST10000NM001G,1027,1
SEAGATE,ST10000NM0086,107143,8
SEAGATE,ST12000NM0007,118330,19
SEAGATE,ST12000NM0008,1815238,73
SEAGATE,ST12000NM001G,1099529,20
SEAGATE,ST12000NM0117,872,1
SEAGATE,ST14000NM0018,2518,null
SEAGATE,ST14000NM001G,966115,24
SEAGATE,ST14000NM0138,144316,17
SEAGATE,ST16000NM001G,1093699,24


There are 10 `SEAGATE` models that have no hard drive failure (`non-functional`) snapshots - represented by `null`.

In [34]:
# A PySpark Dataframe ('q1_2022_seagate_hdd_null_sdf') is created using 
# the PySpark DataFrame ('y2021_fail_seagate_hdd_sdf'), which is 
# grouped by 'manufacturer'and 'model', and a sum is performed.
# The 'sum(functional)' column is renamed to 'functional' and the 
# 'sum(non-functional)' column is renamed to 'non-functional'.
# A filter is applied to only return the `non-functional' with no null
# values.
# Lastly, a sort is performed on the 'model' column in ascending order.
q1_2022_seagate_hdd_null_sdf = (
    q1_2022_seagate_hdd_sdf
    .groupBy('manufacturer', 'model')
    .sum()
    .withColumnRenamed('sum(functional)', 
                       'functional')
    .withColumnRenamed('sum(non-functional)', 
                       'non-functional')
    .filter(F.col('non-functional')
            .isNull())
    .sort('model')
)

In [35]:
q1_2022_seagate_hdd_null_sdf

manufacturer,model,functional,non-functional
SEAGATE,ST14000NM0018,2518,null
SEAGATE,ST16000NM005G,2340,null
SEAGATE,ST18000NM000J,5400,null
SEAGATE,ST4000DM004,505,null
SEAGATE,ST500LM021,2970,null
SEAGATE,ST6000DM001,360,null
SEAGATE,ST6000DM004,90,null
SEAGATE,ST6000DX000,79740,null
SEAGATE,ST8000DM005,2250,null
SEAGATE,ST8000NM000A,4225,null


The 10 `SEAGATE` models that have no hard drive failure (`non-functional`) snapshots, equate to 100,398 `functional` snapshots.  

In [36]:
# The PySpark DataFrame ('q1_2022_seagate_hdd_null_sdf') is grouped 
# by 'manufacturer' and a sum is performed.
# The 'sum(functional)' column is renamed to 'functional' and the 
# 'sum(non-functional)' column is renamed to 'non-functional'.
(
    q1_2022_seagate_hdd_null_sdf
    .groupBy('manufacturer')
    .sum()
    .withColumnRenamed('sum(functional)', 
                       'functional')
    .withColumnRenamed('sum(non-functional)', 
                       'non-functional')
)

manufacturer,functional,non-functional
SEAGATE,100398,null


## Removing SEAGATE HDD models with no (null) hard drive failures

The 10 `SEAGATE` models that have no hard failure (`non-functional`) snapshots are removed to reduce the overall data size and also improve the class imbalance by removing 100,398 snapshots from the majority class (`functional`).  

In [37]:
# A PySpark Dataframe ('q1_2022_seagate_hdd_nonull_sdf') is created 
# using the PySpark DataFrame ('q1_2022_seagate_hdd_sdf'), which is 
# grouped by 'manufacturer'and 'model', and a sum is performed.
# The 'sum(functional)' column is renamed to 'functional' and the 
# 'sum(non-functional)' column is renamed to 'non-functional'.
# A filter is applied to only return the `non-functional' with no null
# values.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'model' column in ascending order.
q1_2022_seagate_hdd_nonull_sdf = (
    q1_2022_seagate_hdd_sdf
    .groupBy('manufacturer', 'model')
    .sum()
    .withColumnRenamed('sum(functional)', 
                       'functional')
    .withColumnRenamed('sum(non-functional)', 
                       'non-functional')
    .filter(F.col('non-functional')
            .isNotNull())
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
    .sort('model')
)

In [38]:
q1_2022_seagate_hdd_nonull_sdf

manufacturer,model,functional,non-functional,failure_percent
SEAGATE,ST10000NM001G,1027,1,0.0973
SEAGATE,ST10000NM0086,107143,8,0.0075
SEAGATE,ST12000NM0007,118330,19,0.0161
SEAGATE,ST12000NM0008,1815238,73,0.004
SEAGATE,ST12000NM001G,1099529,20,0.0018
SEAGATE,ST12000NM0117,872,1,0.1145
SEAGATE,ST14000NM001G,966115,24,0.0025
SEAGATE,ST14000NM0138,144316,17,0.0118
SEAGATE,ST16000NM001G,1093699,24,0.0022
SEAGATE,ST4000DM000,1670278,117,0.007


Although the 10 `SEAGATE` models that have no hard drive failure (`non-functional`) snapshots are removed reducing some of the overall data size, there is minimal impact to the class imbalance.  

In [39]:
# The PySpark Dataframe ('q1_2022_seagate_hdd_nonull_sdf') drops the
# 'failure_percent' column, grouped by 'manufacturer, and a sum is
# performed.
# The 'sum(functional)' column is renamed to 'functional' and the 
# 'sum(non-functional)' column is renamed to 'non-functional'.
# A filter is applied to only return the `non-functional' with no null
# values.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
(
    q1_2022_seagate_hdd_nonull_sdf
    .drop('failure_percent')
    .groupBy('manufacturer')
    .sum()
    .withColumnRenamed('sum(functional)', 
                       'functional')
    .withColumnRenamed('sum(non-functional)', 
                       'non-functional')
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
)

manufacturer,functional,non-functional,failure_percent
SEAGATE,9202845,431,0.0047


## SEAGATE HDD models are selected as the focus for the project

A list is created of the `SEAGATE` `HDD` models that have `functional` and `non-functional` hard drive snapshots. The list of 14 `SEAGATE` `HDD` models will serve as the focus of the project.  

In [40]:
# References
# https://www.geeksforgeeks.org/converting-a-pyspark-dataframe-column-to-a-python-list/
# https://mungingdata.com/pyspark/column-to-list-collect-tolocaliterator/

# A list ('seagate_nonull_list') is created by selecting the models 
# from a PySpark DataFrame ('q1_2022_seagate_hdd_nonull_sdf').
# The list ('seagate_hdd_nonull_list') is sorted and output.
seagate_hdd_nonull_list = (
    list(set
         (q1_2022_seagate_hdd_nonull_sdf
             .select(F.col('model'))
             .toPandas()['model']))
)
seagate_hdd_nonull_list.sort()
seagate_hdd_nonull_list

['ST10000NM001G',
 'ST10000NM0086',
 'ST12000NM0007',
 'ST12000NM0008',
 'ST12000NM001G',
 'ST12000NM0117',
 'ST14000NM001G',
 'ST14000NM0138',
 'ST16000NM001G',
 'ST4000DM000',
 'ST4000DM005',
 'ST500LM030',
 'ST8000DM002',
 'ST8000NM0055']

In [41]:
len(seagate_hdd_nonull_list)

14

Since the previous dataframe was aggregated data and not the hard drive snapshots with the features, a filter is applied using the list of `SEAGATE` `HDD` models that have `functional` and `non-functional` hard drive snapshots to the original dataframe with the `manufacturer` and `model`.  

In [42]:
# References
# https://stackoverflow.com/questions/40421845/pyspark-dataframe-filter-or-include-based-on-list
# https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.Column.isin.html

# A PySpark DataFrame ('q1_2022_seagate_sdf') is created by filtering 
# on the model from the PySpark Data ('q1_2022_split_mfrmod_sdf') when 
# the model is in the list ('seagate_hdd_nonull_list').
# The PySpark DataFrame ('q1_2022_seagate_sdf') is persisted to optimize 
# performance.
q1_2022_seagate_sdf = (
    q1_2022_split_mfrmod_sdf
    .filter(F.col('model')
            .isin(seagate_hdd_nonull_list))
).persist()

In [43]:
# The first 5 rows of the PySpark DataFrame ('q1_2022_seagate_sdf') are
# displayed. The 'persist' is initiated, since 'limit' is an action.
q1_2022_seagate_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,null,null,90,0,100,12,100,0,87,495846641,null,null,89,9937,100,0,null,null,100,12,null,null,null,null,null,null,null,null,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,0,100,0,null,null,66,34,null,null,100,2,99,2641,34,34,null,null,null,null,100,0,100,0,200,0,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,9430,100,75539400520,100,139325292722,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,n

The dataframe consists of 9,203,276 rows and 180 columns. The rows of data have reduced substantially by roughly 51% (from 18,845,260 rows to from 9,203,276 rows) by focusing only on `SEAGATE` as the manufacturer, removing `SSD` models, and removing models with no failures (`non-functional`).  

In [44]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_sdf)

(9203276, 180)

Even with the reduction of roughly 51% rows of data, there is still a signficant class imbalance both at the `model` and `manufacturer` level. In fact, at the model level all hard drive snapshots show a less than 1% failure (0.1145% and less).  

In [45]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is grouped by 
# 'manufacturer' and 'model', a pivot on 'failure', and a count is 
# performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'total' column is created using the 'functional' and
# 'non-functinal' counts.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'model' column in descending 
# order.
(
    q1_2022_seagate_sdf
    .groupby('manufacturer', 'model')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('total', 
                F.col('functional') 
                + F.col('non-functional'))
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
    .sort('model')
)

manufacturer,model,functional,non-functional,total,failure_percent
SEAGATE,ST10000NM001G,1027,1,1028,0.0973
SEAGATE,ST10000NM0086,107143,8,107151,0.0075
SEAGATE,ST12000NM0007,118330,19,118349,0.0161
SEAGATE,ST12000NM0008,1815238,73,1815311,0.004
SEAGATE,ST12000NM001G,1099529,20,1099549,0.0018
SEAGATE,ST12000NM0117,872,1,873,0.1145
SEAGATE,ST14000NM001G,966115,24,966139,0.0025
SEAGATE,ST14000NM0138,144316,17,144333,0.0118
SEAGATE,ST16000NM001G,1093699,24,1093723,0.0022
SEAGATE,ST4000DM000,1670278,117,1670395,0.007


At the manufacturer level, all hard drive snapshots also show less than 1% failure (0.0047%).  

In [46]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count is performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'total' column is created using the 'functional' and
# 'non-functinal' counts.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'model' column in descending 
# order.
(
    q1_2022_seagate_sdf
    .groupby('manufacturer')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('total', 
                F.col('functional') 
                + F.col('non-functional'))
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
)

manufacturer,functional,non-functional,total,failure_percent
SEAGATE,9202845,431,9203276,0.0047


Furthermore, the data has been reduced from 212,404 to 103,713 unique serial drives, which equates to roughly a 51% reduction.  

In [47]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.count_distinct.html

# The PySpark DataFrame ('q1_2022_seagate_sdf') has a count distinct 
# aggregation on 'serial_name' and aliased as 'unique_serial_numbers.
(
    q1_2022_seagate_sdf
    .agg(F.count_distinct('serial_number')
         .alias('unique_serial_numbers'))
)

unique_serial_numbers
103713


For additional validation, the count of hard drive snapshots can be assessed. Since there are 90 days for the first quarter of 2022, no hard drive should have more than 90 snapshots and no hard drive should have less than 1 snapshot. Hard drive with serial number `ZTN0KQXT` has 90 snapshots and hard drive with serial number `WDH1BET6` has 1 snapshot.  

In [48]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is grouped by 
# 'serial_number' and a count is performed.
# Lastly, a sort is performed on the 'model' and 'serial_number' 
# columns in descending order returning the first observation.
(
    q1_2022_seagate_sdf
    .groupby('serial_number')
    .count()
    .sort('count', 'serial_number', 
          ascending=False)
    .first()
)

Row(serial_number='ZTN0KQXT', count=90)

In [49]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is grouped by 
# 'serial_number' and a count is performed.
# Lastly, a sort is performed on the 'model' and 'serial_number' 
# columns in ascending order returning the first observation.
(
    q1_2022_seagate_sdf
    .groupby('serial_number')
    .count()
    .sort('count', 'serial_number')
    .first()
)

Row(serial_number='WDH1BET6', count=1)

## Examine columns and rows with NaN and null values

An examination of the columns reveal that there are 2,156 to 9,203,276 NaN or null (missing) values across multiple columns. For instance, the `smart_1_normalized` and `smart_1_raw` columns indicate 2,156 missing values and the `smart_2_normalized` and `smart_2_raw` columns indicate 9,203,276 missing values. There are 40 columns with 2,156 missing values, 118 columns with 9,203,276 (all) missing values, and 16 columns with missing values that fall within these ranges.  

In [50]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/
# https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_sdf') except for the columns that are date ('date')
# or string ('serial_number', 'model').
# A count is calculated when a NaN or null value is encountered in a
# column to provide a total count of NaN or null values.
(
    q1_2022_seagate_sdf
    .select([F.count(F.when(F.isnan(c) | 
                            F.isnull(c), c))
             .alias(c) 
             for c in q1_2022_seagate_sdf.columns
             if c not in {'date', 'serial_number', 'model'}])
)

capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
0,0,2156,2156,9203276,9203276,2156,2156,2156,2156,2156,2156,2156,2156,9203276,9203276,2156,2156,2156,2156,9203276,9203276,2156,2156,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,4084685,4084685,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,7529675,7529675,5239966,5239966,2156,2156,2156,2156,5239966,5239966,2156,2156,5239966,5239966,2156,2156,2156,2156,2156,2156,4855717,4855717,9203276,9203276,2156,2156,2156,2156,2156,2156,3858340,3858340,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276,9203276

Looking at a percentage perspective with the exception of 0%, the columns appear to range from 42% to 100% missing values. However, the columns with the 2,156 missing values are expressed as 0% due to the small numerator in contrast to the large denominator.    

In [51]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/
# https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/

# The PySpark 'count' function is an expensive computation. 
# The row count of the PySpark DataFrame is calculated once rather
# each time the 'for' loop iterates through the columns.
# The row count is assigned to 'q1_2022_seagate_sdf_rc' variable and 
# used as the demoninator to calculate the percentages of NaN or null 
# values.
q1_2022_seagate_sdf_rc = q1_2022_seagate_sdf.count()

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_sdf') except for the columns that are date ('date')
# or string ('serial_number', 'model').
# A count is calculated when a NaN or null value is encountered in a
# column and the count is divided by the row count denominator 
# ('q1_2022_seagate_sdf_rc') to provide a percentage of NaN or null 
# values.
(
    q1_2022_seagate_sdf
    .select([F.round
             ((F.count
               (F.when(F.isnan(c) | F.isnull(c), c)) 
               / q1_2022_seagate_sdf_rc), 2)
             .alias(c) 
             for c in q1_2022_seagate_sdf.columns 
             if c not in {'date', 'serial_number', 'model'}])
)

capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.44,0.44,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.82,0.82,0.57,0.57,0.0,0.0,0.0,0.0,0.57,0.57,0.0,0.0,0.57,0.57,0.0,0.0,0.0,0.0,0.0,0.0,0.53,0.53,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.42,0.42,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


When inspecting the first column (`smart_1_normalized`) and the third column (`smart_3_normalized`) with 2,156 missing values, the missing values seem to cascade across multiple columns. The same dates and serial numbers of the hard drive snapshots appear makeup the 2,156 missing values.  

In [52]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered to the NaN 
# and null values in the 'smart_1_normalized` column, sorted by 'date, 
# and limited to the first 10 rows.
(
    q1_2022_seagate_sdf
    .filter(F.isnan('smart_1_normalized') |
            F.isnull('smart_1_normalized'))
    .sort('date')
    .limit(10)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-01-01,ZL00555D,ST12000NM0008,-1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-01,ZL00556S,ST12000NM00

In [53]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered to the NaN 
# and null values in the 'smart_3_normalized` column, sorted by 'date, 
# and limited to the first 10 rows.
(
    q1_2022_seagate_sdf
    .filter(F.isnan('smart_3_normalized') |
            F.isnull('smart_3_normalized'))
    .sort('date')
    .limit(10)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-01-01,ZL00555D,ST12000NM0008,-1,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-01,ZL00556S,ST12000NM00

Evaluating the distribution of hard drive snapshots with failures where the `smart_1_normalized` and `smart_3_normalized` values are missing, the distribution looks identical with only 10 `non-functional` hard drive snapshots reflected. Removing these hard drive snapshots will have minimal impact to the minority class (`non-functional`).  

In [54]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered to the NaN 
# and null values in the 'smart_1_normalized` column.
# The '0' values in the 'failure' column are replaced with 'functional'
# and the '1' values are replaced with 'non-functional' using a regular
# expression replace.
# The PySpark DataFrame ('q1_2022_seagate_sdf') grouped by 'failure' 
# and a count is performed.
(
    q1_2022_seagate_sdf
    .filter(F.isnan('smart_1_normalized') |
            F.isnull('smart_1_normalized'))
    .withColumn('failure', 
                F.regexp_replace('failure', '0', 'functional'))
    .withColumn('failure', 
                F.regexp_replace('failure', '1', 'non-functional'))
    .groupBy('failure')
    .count()
)

failure,count
functional,2146
non-functional,10


In [55]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered to the NaN 
# and null values in the 'smart_3_normalized` column.
# The '0' values in the 'failure' column are replaced with 'functional'
# and the '1' values are replaced with 'non-functional' using a regular
# expression replace.
# The PySpark DataFrame ('y2021_seagate_sdf') grouped by 'failure' and
# a count is performed.
(
    q1_2022_seagate_sdf
    .filter(F.isnan('smart_3_normalized') |
            F.isnull('smart_3_normalized'))
    .withColumn('failure', 
                F.regexp_replace('failure', '0', 'functional'))
    .withColumn('failure', 
                F.regexp_replace('failure', '1', 'non-functional'))
    .groupBy('failure')
    .count()
)

failure,count
functional,2146
non-functional,10


Closer inspection of the snapshots with failures and missing values for the `smart_1_normalized` and `smart_3_normalized` columns reveal the snapshots are similar.

In [56]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered where the
# 'failure' column is '1' and 'smart_1_normalized' column is null, then
# Lastly, a sort is performed on the 'serial_number' column in 
# ascending order returning the first 5 rows.
(
    q1_2022_seagate_sdf
    .filter((F.col('failure') == "1") &
            (F.isnull('smart_1_normalized')))
    .sort('serial_number')
    .limit(5)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-03-29,ZHZ2RS53,ST14000NM0138,14000519643136,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-02,ZHZ3T1AC

In [57]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered where the
# 'failure' column is '1' and 'smart_3_normalized' column is null, then
# Lastly, a sort is performed on the 'serial_number' column in 
# ascending order returning the first 5 rows.
(
    q1_2022_seagate_sdf
    .filter((F.col('failure') == "1") &
            (F.isnull('smart_3_normalized')))
    .sort('serial_number')
    .limit(5)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-03-29,ZHZ2RS53,ST14000NM0138,14000519643136,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-02,ZHZ3T1AC

Randomly selecting the output of the first and last serial numbers (ZHZ2RS53 and ZJV3CKGR) from the `serial_number` column, sorted by date, there is not an apparent opportunity for imputing the missing values from a future or past snapshot.  

In [58]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered where the
# 'serial_number' column is 'ZHZ2RS53'.
# Lastly, a sort is performed on the 'date' column in ascending order 
# returning the first 5 rows.
(
    q1_2022_seagate_sdf
    .filter(F.col('serial_number') == "ZHZ2RS53")
    .sort('date')
    .limit(5)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-01-01,ZHZ2RS53,ST14000NM0138,14000519643136,0,81,115457728,null,null,90,0,100,24,100,0,86,445457700,null,null,89,9807,100,0,null,null,100,22,null,null,null,null,null,null,null,null,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,0,100,0,null,null,69,31,null,null,100,45,100,1254,31,31,1,115457728,null,null,100,0,100,0,200,0,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,8463,100,70162834995,100,109397224959,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-02,ZHZ2RS53,ST14000NM0138,14000519643136,0,84,229019

In [59]:
# The PySpark DataFrame ('q1_2022_seagate_sdf') is filtered where the
# 'serial_number' column is 'ZJV3CKGR'.
# Lastly, a sort is performed on the 'date' column in ascending order 
# returning the first 5 rows.
(
q1_2022_seagate_sdf.
    filter(F.col('serial_number') == "ZJV3CKGR")
    .sort('date')
    .limit(5)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,smart_164_raw,smart_165_normalized,smart_165_raw,smart_166_normalized,smart_166_raw,smart_167_normalized,smart_167_raw,smart_168_normalized,smart_168_raw,smart_169_normalized,smart_169_raw,smart_170_normalized,smart_170_raw,smart_171_normalized,smart_171_raw,smart_172_normalized,smart_172_raw,smart_173_normalized,smart_173_raw,smart_174_normalized,smart_174_raw,smart_175_normalized,smart_175_raw,smart_176_normalized,smart_176_raw,smart_177_normalized,smart_177_raw,smart_178_normalized,smart_178_raw,smart_179_normalized,smart_179_raw,smart_180_normalized,smart_180_raw,smart_181_normalized,smart_181_raw,smart_182_normalized,smart_182_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_202_normalized,smart_202_raw,smart_206_normalized,smart_206_raw,smart_210_normalized,smart_210_raw,smart_218_normalized,smart_218_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_230_normalized,smart_230_raw,smart_231_normalized,smart_231_raw,smart_232_normalized,smart_232_raw,smart_233_normalized,smart_233_raw,smart_234_normalized,smart_234_raw,smart_235_normalized,smart_235_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_244_normalized,smart_244_raw,smart_245_normalized,smart_245_raw,smart_246_normalized,smart_246_raw,smart_247_normalized,smart_247_raw,smart_248_normalized,smart_248_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw,manufacturer
2022-01-01,ZJV3CKGR,ST12000NM0007,12000138625024,0,80,102053768,null,null,91,0,100,7,100,0,90,918279375,null,null,78,20033,100,0,null,null,100,7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,0,100,4295360572,null,null,70,30,null,null,100,268,100,1149,30,30,80,102053768,null,null,100,0,100,0,200,100,100,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,100,19854,100,80810888384,100,279643202618,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,SEAGATE
2022-01-02,ZJV3CKGR,ST12000NM0007,120001386

## Drop columns with missing values greater than 40%

There are differing perspectives on the acceptable percentage of missing values in columns and rows to drop. Since there are a substantial amount of columns (179 potential columns or features not including the target  (`failure`)), a column will be dropped if there are 40% or more missing values. The intent of dropping these columns is to improve feature importance and reduce dimensionality.  

In [60]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/
# https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/

# The PySpark 'count' function is an expensive computation. 
# The row count of the PySpark DataFrame is calculated once rather
# each time the 'for' loop iterates through the columns.
# The row count is assigned to 'q1_2022_seagate_sdf_rc' variable and 
# used as the demoninator to calculate the percentages of NaN or null 
# values.
q1_2022_seagate_sdf_rc = q1_2022_seagate_sdf.count()

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_sdf') except for the columns that are date ('date')
# or string ('serial_number', 'model').
# A count is calculated when a NaN or null value is encountered in a
# column and the count is divided by the row count denominator 
# ('q1_2022_seagate_sdf_rc') to provide a percentage of NaN or null 
# values. The results are collected in a list ('nannull_col_pct_list').
nannull_col_pct_list = (
    q1_2022_seagate_sdf
    .select([F.round
             ((F.count
               (F.when(F.isnan(c) | F.isnull(c), c))
               / q1_2022_seagate_sdf_rc), 2)
             .alias(c)
             for c in q1_2022_seagate_sdf.columns 
             if c not in {'date', 'serial_number', 'model'}]).collect()
)

In [61]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/

# A list comprehension is used to iterate over a list 
# ('nannull_col_pct_list') and converts the row to a dictionary within 
# a list and is assigned to the 'nannull_col_pct_dict_list' variable.
nannull_col_pct_dict_list = [row.asDict() \
                             for row in nannull_col_pct_list]

# A dictionary comprehension is used to iterate over the keys, values  
# in the nested dictionary ('nannull_col_pct_dict_list[0]') and if the
# value ('v') is greater than 0.4 (40%), the key ('k') is collected in 
# a list ('nannull_col'). 
nannull_col = list({k for (k, v) \
                    in nannull_col_pct_dict_list[0].items() \
                    if v > 0.4})

The quantity of columns or features decreases from 180 to 46 by dropping the columns with 40% or more missing values. The column or target (`failure`) is not impacted.  

In [62]:
# Display the quantity of columns before the columns are dropped.
len(q1_2022_seagate_sdf.columns)

180

In [63]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/

# The list of columns with NaN or null values greater than 0.4 or 40% 
# ('nannull_col') are dropped from the PySpark DataFrame 
# ('q1_2022_seagate_sdf') and are saved to a PySpark DataFrame
# ('q1_2022_seagate_dropnacol_sdf').

q1_2022_seagate_dropnacol_sdf = (
    q1_2022_seagate_sdf
    .drop(*nannull_col)
)

In [64]:
# Display the quantity of columns after the columns are dropped.
len(q1_2022_seagate_dropnacol_sdf.columns)

46

In [65]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_seagate_dropnacol_sdf') are displayed.
q1_2022_seagate_dropnacol_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,manufacturer
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,90,0,100,12,100,0,87,495846641,89,9937,100,0,100,12,100,0,100,0,66,34,100,2,99,2641,34,34,100,0,100,0,200,0,100,9430,100,75539400520,100,139325292722,SEAGATE
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,99,0,100,1,100,0,90,905087530,85,13174,100,0,100,1,100,0,100,0,53,47,100,0,99,3840,47,47,100,0,100,0,200,0,100,12539,100,70240933168,100,164363287720,SEAGATE
2022-01-01,ZA1FLE1P,ST8000NM0055,8001563222016,0,82,157857120,91,0,100,7,100,0,93,1858834907,86,12781,100,0,100,7,100,0,100,0,63,37,100,105,100,693,37,37,100,0,100,0,200,0,100,12734,100,25301521848,100,173854085302,SEAGATE
2022-01-01,ZA16NQJR,ST8000NM0055,8001563222016,0,84,234265456,89,0,100,8,100,0,95,2891515537,54,40970,100,0,100,8,100,0,100,0,67,33,100,15,69,62343,33,33,100,0,100,0,200,0,100,40936,100,83209739680,100,302120432268,SEAGATE
2022-01-01,ZA130TTW,ST8000DM002,8001563222016,0,83,221445736,96,0,100,2,100,0,94,2740044896,48,46053,100,0,100,2,100,0,100,0,70,30,100,0,74,52477,30,30,100,0,100,0,200,0,100,46007,100,92979000192,100,334927052596,SEAGATE


## Drop rows with missing values greater than 40%

After dropping the columns with 40% or more missing values there are still missing values to address. There are still 40 columns or features with 2,156 values missing. As previously determined, there are rows or observations of data with multiple null values in each column. Rather than dropping a column for such a miniscule amount of missing values, the rows or observations will be dropped with 40% or more missing values.  

In [66]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/
# https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_dropnacol_sdf') except for the columns that are 
# date ('date') or string ('serial_number', 'model').
# A count is calculated when a NaN or null value is encountered in a
# column to provide a total count of NaN or null values.
(
    q1_2022_seagate_dropnacol_sdf
    .select([F.count
             (F.when(F.isnan(c) | F.isnull(c), c))
             .alias(c)
             for c in q1_2022_seagate_dropnacol_sdf.columns
             if c not in {'date', 'serial_number', 'model'}])
)

capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,manufacturer
0,0,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,2156,0


There are 9,203,276 rows prior to dropping the 2,156 missing values.    

In [67]:
# The PySpark DataFrame ('q1_2022_seagate_dropnacol_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count is performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'total' column is created using the 'functional' and
# 'non-functinal' counts.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_seagate_dropnacol_sdf
    .groupby('manufacturer')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('total', F.col('functional') 
                + F.col('non-functional'))
    .withColumn('failure_percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
)

manufacturer,functional,non-functional,total,failure_percent
SEAGATE,9202845,431,9203276,0.0047


If a row meets or exceeds the threshold of missing values in 18 (40% for 46 columns) or more columns, the row will be dropped.  

In [68]:
# Amount of columns for the PySpark DataFrame 
# (q1_2022_seagate_dropnacol_sdf).
len(q1_2022_seagate_dropnacol_sdf.columns)

46

In [69]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.dropna.html

# Establish a threshold of columns ('thresh') based on 0.4 or 40%.
# Based on 46 columns, 0.4 or 40% columns equate to 18 columns.
thresh = round(len(q1_2022_seagate_dropnacol_sdf.columns) * 0.4)
print(thresh)

18


In [70]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.dropna.html

# Using the threshold of 0.4 or 40% ('thresh'), rows are dropped 
# if 18 or more columns have missing values.
q1_2022_seagate_dropnarow_sdf = q1_2022_seagate_dropnacol_sdf.dropna(thresh=thresh)

Dropping the 2,156 missing values results in 9,201,120 rows, results in a maximum impact (a reduction of 2,146 rows or observations) on the majority class (`functional`) and a minimal impact (a reduction of 10 rows or observations) on the minority class (`non-functional`). The effort assists with balancing the classes. Once the rows are dropped there are no missing values.  

In [71]:
# The PySpark DataFrame ('q1_2022_seagate_dropnarow_sdf') is grouped by 
# 'manufacturer', a pivot on 'failure', and a count is performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# A 'total' column is created using the 'functional' and
# 'non-functinal' counts.
# A 'failure_percent' column is created and rounded (to 4 places) using 
# the 'non-functional' counts divided by the 'functional' and 
# 'non-functional' counts multiplied by 100.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_seagate_dropnarow_sdf
    .groupby('manufacturer')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .withColumn('total', 
                F.col('functional') 
                + F.col('non-functional'))
    .withColumn('failure percent', 
                F.round(F.col('non-functional') 
                        / (F.col('functional') 
                           + F.col('non-functional'))  
                        * 100, 4))
)

manufacturer,functional,non-functional,total,failure percent
SEAGATE,9200699,421,9201120,0.0046


In [72]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_seagate_dropnarow_sdf') are displayed.
q1_2022_seagate_dropnarow_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,manufacturer
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,90,0,100,12,100,0,87,495846641,89,9937,100,0,100,12,100,0,100,0,66,34,100,2,99,2641,34,34,100,0,100,0,200,0,100,9430,100,75539400520,100,139325292722,SEAGATE
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,99,0,100,1,100,0,90,905087530,85,13174,100,0,100,1,100,0,100,0,53,47,100,0,99,3840,47,47,100,0,100,0,200,0,100,12539,100,70240933168,100,164363287720,SEAGATE
2022-01-01,ZA1FLE1P,ST8000NM0055,8001563222016,0,82,157857120,91,0,100,7,100,0,93,1858834907,86,12781,100,0,100,7,100,0,100,0,63,37,100,105,100,693,37,37,100,0,100,0,200,0,100,12734,100,25301521848,100,173854085302,SEAGATE
2022-01-01,ZA16NQJR,ST8000NM0055,8001563222016,0,84,234265456,89,0,100,8,100,0,95,2891515537,54,40970,100,0,100,8,100,0,100,0,67,33,100,15,69,62343,33,33,100,0,100,0,200,0,100,40936,100,83209739680,100,302120432268,SEAGATE
2022-01-01,ZA130TTW,ST8000DM002,8001563222016,0,83,221445736,96,0,100,2,100,0,94,2740044896,48,46053,100,0,100,2,100,0,100,0,70,30,100,0,74,52477,30,30,100,0,100,0,200,0,100,46007,100,92979000192,100,334927052596,SEAGATE


Dropping the rows with missing values in 18 or more columns addresses the remaining missing values. Each column now reports 0 missing values.

In [73]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/
# https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_dropnarow_sdf') except for the columns that are
# date ('date') or string ('serial_number', 'model').
# A count is calculated when a NaN or null value is encountered in a
# column to provide a total count of NaN or null values.
(
    q1_2022_seagate_dropnarow_sdf
    .select([F.count
             (F.when(F.isnan(c) | F.isnull(c), c))
             .alias(c)
             for c in q1_2022_seagate_dropnarow_sdf.columns
             if c not in {'date', 'serial_number', 'model'}])
)

capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,manufacturer
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_dropnarow_sdf)

(9201120, 46)

## Examine columns to determine if the values are the distinct

Having a column with a single value adds no predictive value to the data. For example, the `smart_3_raw` and `smart_10_raw` columns or features are entirely populated with the value of `0`. 

In [75]:
# References
# https://stackoverflow.com/questions/53808602/pyspark-drop-columns-that-have-same-values-in-all-rows
# https://stackoverflow.com/questions/53763417/number-of-unique-elements-in-all-columns-of-a-pyspark-dataframe
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.approx_count_distinct.html

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_dropnarow_sdf').
# An aggregation is performed on each column to obtain an approximate 
# distinct count of column. An approximate distinct count is chosen
# over a distinct count due to the amount of columns and computational
# expense, resulting in 10 times reduction in time.
(
    q1_2022_seagate_dropnarow_sdf
    .agg(*(F.approx_count_distinct
           (F.col(c)).alias(c)
           for c in q1_2022_seagate_dropnarow_sdf.columns))
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,manufacturer
86,101277,14,7,2,83,8327387,18,1,7,298,100,4537,45,9027043,76,63172,2,1,4,219,103,629,20,2124,48,50,8,1133,94,88001,50,50,46,440,46,440,7,357,1,62186,1,8512965,1,8536760,1


All the values in the `smart_3_raw` column are `0`.

In [76]:
# The PySpark DataFrame ('q1_2022_seagate_dropnarow_sdf') is grouped by 
# 'smart_3_raw' and a count is performed.
(
    q1_2022_seagate_dropnarow_sdf
    .groupBy('smart_3_raw')
    .count()
)

smart_3_raw,count
0,9201120


All the values in the `smart_10_raw` column are `0`.

In [77]:
# The PySpark DataFrame ('q1_2022_seagate_dropnarow_sdf') is grouped by 
# 'smart_10_raw' and a count is performed.
(
    q1_2022_seagate_dropnarow_sdf
    .groupBy('smart_10_raw')
    .count()
)

smart_10_raw,count
0,9201120


In [78]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/
# https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_dropnarow_sdf').
# An aggregation is performed on each column to obtain an approximate 
# distinct count of column.
# The results are collected in a list 
# ('q1_2022_seagate_distinct_list').
q1_2022_seagate_distinct_list = (
    q1_2022_seagate_dropnarow_sdf
    .agg(*(F.approx_count_distinct
           (F.col(c)).alias(c)
           for c in q1_2022_seagate_dropnarow_sdf.columns))
    .collect()
)

There are 6 columns or features that have only 1 distinct value in the column. This includes the `manufacturer` column, which is entirely populated with the value of `SEAGATE` (the hard drive manufacturer being analyzed).  

In [79]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/

# A list comprehension is used to iterate over a 
# list ('q1_2022_seagate_distinct_list') and converts the
# row to a dictionary within a list and is assigned to
# the 'distinct_col_dict_list' variable.
distinct_col_dict_list = [row.asDict() \
                          for row in q1_2022_seagate_distinct_list]

# A dictionary comprehension is used to iterate over the keys, values 
# in the nested dictionary ('distinct_col_dict_list[0]') and if the
# value ('v') is less than 1, the key ('k') is collected in a 
# list ('nondistinctcol'). 
nondistinctcol = list({k for (k, v) \
                       in distinct_col_dict_list[0].items() \
                       if v <= 1})

nondistinctcol.sort()
nondistinctcol

['manufacturer',
 'smart_10_raw',
 'smart_240_normalized',
 'smart_241_normalized',
 'smart_242_normalized',
 'smart_3_raw']

The quantity of columns or features decreases 6 columns from 46 to 40 (including the target (`failure`)) by dropping the columns with only 1 distinct value.  

In [80]:
# Display the quantity of columns before the columns are
# dropped.
len(q1_2022_seagate_dropnarow_sdf.columns)

46

In [81]:
# References:
# https://www.coursera.org/learn/clean-explore-visualize-big-data-python-spark/

# The list of columns with NaN or null values greater than 
# 0.4 or 40% ('nannull_col') are dropped from the PySpark DataFrame.
q1_2022_seagate_nondistinct_sdf = q1_2022_seagate_dropnarow_sdf.drop(*nondistinctcol)

In [82]:
# Display the quantity of columns after the columns are
# dropped.
len(q1_2022_seagate_nondistinct_sdf.columns)

40

In [83]:
# References
# https://stackoverflow.com/questions/53808602/pyspark-drop-columns-that-have-same-values-in-all-rows
# https://stackoverflow.com/questions/53763417/number-of-unique-elements-in-all-columns-of-a-pyspark-dataframe
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.approx_count_distinct.html

# A 'for' loop iterates through each column in the PySpark DataFrame
# ('q1_2022_seagate_nondistinct_sdf').
# An aggregation is performed on each column to obtain an approximate 
# distinct count of column.
(
    q1_2022_seagate_nondistinct_sdf
    .agg(*(F.approx_count_distinct
           (F.col(c)).alias(c) 
           for c in q1_2022_seagate_nondistinct_sdf.columns))
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
86,101277,14,7,2,83,8327387,18,7,298,100,4537,45,9027043,76,63172,2,4,219,103,629,20,2124,48,50,8,1133,94,88001,50,50,46,440,46,440,7,357,62186,8512965,8536760


In [84]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_seagate_nondistinct_sdf') are displayed.
q1_2022_seagate_nondistinct_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,90,100,12,100,0,87,495846641,89,9937,100,100,12,100,0,100,0,66,34,100,2,99,2641,34,34,100,0,100,0,200,0,9430,75539400520,139325292722
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,99,100,1,100,0,90,905087530,85,13174,100,100,1,100,0,100,0,53,47,100,0,99,3840,47,47,100,0,100,0,200,0,12539,70240933168,164363287720
2022-01-01,ZA1FLE1P,ST8000NM0055,8001563222016,0,82,157857120,91,100,7,100,0,93,1858834907,86,12781,100,100,7,100,0,100,0,63,37,100,105,100,693,37,37,100,0,100,0,200,0,12734,25301521848,173854085302
2022-01-01,ZA16NQJR,ST8000NM0055,8001563222016,0,84,234265456,89,100,8,100,0,95,2891515537,54,40970,100,100,8,100,0,100,0,67,33,100,15,69,62343,33,33,100,0,100,0,200,0,40936,83209739680,302120432268
2022-01-01,ZA130TTW,ST8000DM002,8001563222016,0,83,221445736,96,100,2,100,0,94,2740044896,48,46053,100,100,2,100,0,100,0,70,30,100,0,74,52477,30,30,100,0,100,0,200,0,46007,92979000192,334927052596


## Create deteriorating window data

Predicting a failure at the time of failure offers little benefit toward predictive maintenance. A strategy is needed to create a window of hard drive snapshots leading up to the failure. A literature review revealed varying attempts at creating a deteriorating window. Xu et al. (2016) intuitively created 6 levels of health degrees, where Level 1 represented a "red alert" and a remaining time of life is less than 72 hours.  Pinciroli et al. (2020) chose 4 "lookahead windows" - 0 days, 1 day, 2 days, and 7 days, based on probability of uncorrectable errors. However, Pinciroli et al. (2020) acknowledged that the probability of uncorrectable errors was much higher and noticeable at 2 days compared to 7 days. Research by Shen et al. (2021) identified "soon-to-fail drives are predicted within 7 days" using a long short-term memory (LSTM) recurrent neural network (RNN). Based on these findings, a decision was made to implement a deteriorating window of 7 days (6 days leading to the failure and the day of the failure). Evaluating hard drive snapshots 7 days prior to failure will also mitigate class imbalance. Depending on business need, hardware redundancy, reliability standards (i.e. 99.999% availability), or resource and staffing, data centers might have different requirements for a deteriorating window.  

The approach consists of splitting the dataframe into two - `functional` and `non-functional` hard drive snapshots.  
<br>
The `non-functional` hard drive snapshots dataframe will remain classified as `non-functional` for the target feature.  
<br>
Using serial numbers of failed hard drives, the `functional` hard drive snapshots are further split into two dataframes - a dataframe of `functional` hard drive snapshots that eventually experience a failure and a dataframe of `functional` hard drives snapshots that never experienced a failure in the first quarter of 2022.  
<br>
From the  dataframe of `functional` hard drive snapshots that eventually experience a failure, the last 6 days of hard drive snapshots prior to a failure are re-classified as `non-functional` for the target feature. This equates to 7 days of hard drive snapshots per failed hard drive - 6 days leading to the failure and the last day the failure occurred. This assists with the class imbalance by adding more hard drive snapshots (up to 6 additional per hard drive) to the minority class (`non-functional`).  
<br>
The `functional` hard drive snapshots that eventually experience a failure (outside the 6 days) and the `functional` hard drives snapshots that never experienced a failure in the first quarter of 2022 remain classified as `functional` for the target.  

A list of serial numbers is created for hard drives that experienced a hard drive failure.  

In [85]:
# References
# https://www.geeksforgeeks.org/converting-a-pyspark-dataframe-column-to-a-python-list/
# https://mungingdata.com/pyspark/column-to-list-collect-tolocaliterator/

# Select serial numbers and filter on failures from the PySpark 
# DataFrame ('q1_2022_seagate_nondistinct_sdf') to a Pandas Series.
# From the Pandas Series, create a list ('seagate_sn_fail_list) of 
# unique serial numbers.
seagate_sn_fail_list = list(set(q1_2022_seagate_nondistinct_sdf
                                .select(F.col('serial_number'))
                                .filter(F.col('failure') == "1")
                                .toPandas()['serial_number']))

seagate_sn_fail_list.sort()
seagate_sn_fail_list[:5]

['S300Z5M7', 'S300Z79M', 'S300Z7WQ', 'S300ZAFS', 'S3010LL5']

There are 421 serial numbers associated with a hard drive failure.  

In [86]:
# The quantity of unique serial numbers with failures in the list
# ('seagate_sn_fail_list') is displayed.
len(seagate_sn_fail_list)

421

Prior to conducting transformations, a count is documented of the `functional` and `non-functional` hard drive snapshots to ensure the same amount after the transformations.  There are 9,200,699 `functional` and 421 `non-functional` hard drive snapshots.  

In [87]:
# The PySpark DataFrame ('q1_2022_seagate_nondistinct_sdf') '0' values 
# in the 'failure' column are replaced with 'functional' and the '1' 
# values are replaced with 'non-functional' using a regular expression
# replace.
# The PySpark DataFrame ('q1_2022_seagate_nondistinct_sdf') grouped by
# 'failure' and a count is performed.
(
    q1_2022_seagate_nondistinct_sdf
    .withColumn('failure', 
                F.regexp_replace('failure', '0', 'functional'))
    .withColumn('failure', 
                F.regexp_replace('failure', '1', 'non-functional'))
    .groupBy('failure')
    .count()
)

failure,count
functional,9200699
non-functional,421


A total of 9,201,120 hard drive snapshots.  

In [88]:
q1_2022_seagate_nondistinct_sdf.count()

9201120

The dataframe is split into two dataframes - one for failure and one without failures.

In [89]:
# A PySpark DataFrame ('q1_2022_seagate_fail_sdf') is created by
# filtering a PySpark DataFrame ('q1_2022_seagate_nondistinct_sdf')
# where there is a failure.
q1_2022_seagate_fail_sdf = (
    q1_2022_seagate_nondistinct_sdf
    .filter(F.col('failure') == "1")
)

There are 421 hard drive snapshots with failures.  

In [90]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_fail_sdf)

(421, 40)

In [91]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_seagate_fail_sdf') are displayed.
q1_2022_seagate_fail_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-04,Z304KCKY,ST4000DM000,4000787030016,1,115,92563448,91,100,18,100,0,89,822020539,39,53738,100,100,18,43,57,100,0,77,23,100,1,96,8654,23,23,100,56,100,56,200,0,53676,59931366448,401183281282
2022-01-02,ZHZ4YA04,ST12000NM0008,12000138625024,1,79,70963736,90,100,11,100,3408,84,279190680,80,18132,100,100,10,1,176,100,131074,62,38,100,3,100,1894,38,38,100,96,100,96,200,0,17662,68554808168,243284577040
2022-01-02,ZHZ3PXX8,ST12000NM0008,12000138625024,1,70,94032960,97,100,3,100,928,87,503737847,79,19183,100,100,3,27,73,100,4296802332,75,25,100,219,100,997,25,25,100,376,100,376,200,0,19023,81545451064,193939767296
2022-01-03,S300Z5M7,ST4000DM000,4000787030016,1,118,197381016,93,100,7,100,0,87,591787714,34,58164,100,100,7,100,0,100,0,77,23,100,0,79,42224,23,23,100,0,100,0,200,0,57997,70392335232,351282420987
2022-01-04,Z302F1LL,ST4000DM000,4000787030016,1,119,217883536,94,100,6,100,296,88,751934402,31,60546,100,100,6,100,0,100,0,81,19,100,0,75,50982,19,19,100,0,100,0,200,0,60366,74075551128,560293504225


The table indicates the models and counts of the hard drive snapshots of hard drives that result in failure.  

In [92]:
# The PySpark DataFrame ('q1_2022_seagate_fail_sdf') is grouped by 
# 'model', a pivot on 'failure', a count is performed. 
# The '1' column is renamed to 'non-functional'.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_seagate_fail_sdf
    .groupby('model')
    .pivot('failure')
    .count()
    .withColumnRenamed('1', 'non-functional')
    .sort('model')
)

model,non-functional
ST10000NM001G,1
ST10000NM0086,8
ST12000NM0007,18
ST12000NM0008,70
ST12000NM001G,19
ST12000NM0117,1
ST14000NM001G,22
ST14000NM0138,16
ST16000NM001G,22
ST4000DM000,117


In [93]:
# A PySpark DataFrame ('q1_2022_seagate_nofail_sdf') is created by
# filtering a PySpark DataFrame ('q1_2022_seagate_nondistinct_sdf') 
# where there is no failure.
q1_2022_seagate_nofail_sdf = (
    q1_2022_seagate_nondistinct_sdf
    .filter(F.col('failure') == "0")
)

There are 9,200,699 hard drive snapshots without failures.  

In [94]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_nofail_sdf)

(9200699, 40)

In [95]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_seagate_nofail_sdf') are displayed.
q1_2022_seagate_nofail_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,90,100,12,100,0,87,495846641,89,9937,100,100,12,100,0,100,0,66,34,100,2,99,2641,34,34,100,0,100,0,200,0,9430,75539400520,139325292722
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,99,100,1,100,0,90,905087530,85,13174,100,100,1,100,0,100,0,53,47,100,0,99,3840,47,47,100,0,100,0,200,0,12539,70240933168,164363287720
2022-01-01,ZA1FLE1P,ST8000NM0055,8001563222016,0,82,157857120,91,100,7,100,0,93,1858834907,86,12781,100,100,7,100,0,100,0,63,37,100,105,100,693,37,37,100,0,100,0,200,0,12734,25301521848,173854085302
2022-01-01,ZA16NQJR,ST8000NM0055,8001563222016,0,84,234265456,89,100,8,100,0,95,2891515537,54,40970,100,100,8,100,0,100,0,67,33,100,15,69,62343,33,33,100,0,100,0,200,0,40936,83209739680,302120432268
2022-01-01,ZA130TTW,ST8000DM002,8001563222016,0,83,221445736,96,100,2,100,0,94,2740044896,48,46053,100,100,2,100,0,100,0,70,30,100,0,74,52477,30,30,100,0,100,0,200,0,46007,92979000192,334927052596


The table indicates the models and counts of the hard drive snapshots of hard drives that never experience a failure in the first quarter of 2022.   

In [96]:
# The PySpark DataFrame ('q1_2022_seagate_nofail_sdf') is grouped by 
# 'model', a pivot on 'failure', a count is performed. 
# The '0' column is renamed to 'functional'.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_seagate_nofail_sdf
    .groupby('model')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .sort('model')
)

model,functional
ST10000NM001G,1027
ST10000NM0086,107118
ST12000NM0007,118328
ST12000NM0008,1815146
ST12000NM001G,1099523
ST12000NM0117,872
ST14000NM001G,966104
ST14000NM0138,144312
ST16000NM001G,1092329
ST4000DM000,1670153


The data of hard drive snapshots without a failure is further split. Using the list of serial numbers of hard drives that failed, a dataframe is composed of hard drive snapshots that are operational but *eventually* experienced a failure. The dataframe will be used to capture the 6 days of data hard drive snapshots prior to the failure. These 6 days of data will be re-classified as hard drive snapshots with a failure (i.e. `non-functional` or `1`).

In [97]:
# References
# https://stackoverflow.com/questions/40421845/pyspark-dataframe-filter-or-include-based-on-list
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.isin.html

# A PySpark DataFrame ('q1_2022_seagate_prefail_sdf') is created from 
# the PySpark DataFrame ('y2021_seagate_nofail_sdf') after a filter on
# 'serial_number' is applied to only obtain observations in the list
# ('seagate_sn_fail_list').
q1_2022_seagate_prefail_sdf = (
    q1_2022_seagate_nofail_sdf
    .filter(F.col('serial_number')
            .isin(seagate_sn_fail_list))
)

There are 18,971 hard drive snapshots that are operational but *eventually* experienced a failure.  

In [98]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_prefail_sdf)

(18971, 40)

In [99]:
# The first 5 rows of the PySpark DataFrame 
# ('y2021_seagate_prefail_sdf') are displayed.
q1_2022_seagate_prefail_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-01,ZHZ5YD7V,ST12000NM0008,12000138625024,0,75,34667744,95,100,4,100,0,85,282923953,82,16166,100,100,4,100,0,100,0,70,30,100,0,99,2052,30,30,100,0,100,0,200,0,15812,65604427712,319441376130
2022-01-01,Z304KR32,ST4000DM000,4000787030016,0,103,99543881,91,100,19,99,1560,88,804640358,38,54645,100,100,19,1,241,100,0,81,19,100,2,96,9640,19,19,98,464,98,464,200,0,54543,68902126856,320399440634
2022-01-01,ZLW17SAP,ST14000NM001G,14000519643136,0,82,168514160,97,100,3,100,0,89,770112624,88,11261,100,100,3,100,0,100,0,75,25,100,1,100,864,25,25,100,0,100,0,200,0,11054,77979075480,164377694459
2022-01-01,ZL005220,ST12000NM0008,12000138625024,0,84,234696440,91,100,9,100,0,80,92609998,90,9128,100,100,9,100,0,100,0,68,32,100,3,100,1038,32,32,100,0,100,0,200,0,8903,50091949856,151057816808
2022-01-01,S301KX68,ST4000DM000,4000787030016,0,117,119498952,91,100,27,100,0,84,281512390,38,54661,100,100,27,94,6,100,0,78,22,100,4,97,7452,22,22,100,0,100,0,200,0,54560,65360576152,421137921727


The table indicates the models and counts of the hard drive snapshots of hard drives that *eventually* result in failure.  

In [100]:
# The PySpark DataFrame ('q1_2022_seagate_prefail_sdf') is grouped by
# 'model' and a count is performed.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_seagate_prefail_sdf
    .groupBy('model')
    .count()
    .sort('model')
)

model,count
ST10000NM0086,510
ST12000NM0007,836
ST12000NM0008,3322
ST12000NM001G,786
ST12000NM0117,62
ST14000NM001G,857
ST14000NM0138,782
ST16000NM001G,790
ST4000DM000,5649
ST500LM030,29


The data of hard drive snapshots without a failure is further split. Using the list of serial numbers of hard drives that failed, a dataframe is composed of hard drive snapshots that are operational and *never* experienced a failure in the first quarter of 2022. These hard drive snapshots will remain classified as no failure (i.e. `functional` or `0`).

In [101]:
# References
# https://stackoverflow.com/questions/40421845/pyspark-dataframe-filter-or-include-based-on-list
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.isin.html

# A PySpark DataFrame ('q1_2022_seagate_running_sdf') is created from 
# the PySpark DataFrame ('q1_2022_seagate_nofail_sdf') after a filter 
# on 'serial_number' is applied to exclude observations in the list
# ('seagate_sn_fail_list').
q1_2022_seagate_running_sdf = (
    q1_2022_seagate_nofail_sdf
    .filter(~F.col('serial_number')
            .isin(seagate_sn_fail_list))
)

There are 9,181,728 hard drive snapshots that are operational and *never* experienced a failure in the first quarter of 2022.  

In [102]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_running_sdf)

(9181728, 40)

In [103]:
# The first 5 rows of the PySpark DataFrame 
# ('q1_2022_seagate_running_sdf') are displayed.
q1_2022_seagate_running_sdf.limit(5)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-01,ZLW18P9K,ST14000NM001G,14000519643136,0,73,20467240,90,100,12,100,0,87,495846641,89,9937,100,100,12,100,0,100,0,66,34,100,2,99,2641,34,34,100,0,100,0,200,0,9430,75539400520,139325292722
2022-01-01,ZLW0EGC7,ST12000NM001G,12000138625024,0,84,228715872,99,100,1,100,0,90,905087530,85,13174,100,100,1,100,0,100,0,53,47,100,0,99,3840,47,47,100,0,100,0,200,0,12539,70240933168,164363287720
2022-01-01,ZA1FLE1P,ST8000NM0055,8001563222016,0,82,157857120,91,100,7,100,0,93,1858834907,86,12781,100,100,7,100,0,100,0,63,37,100,105,100,693,37,37,100,0,100,0,200,0,12734,25301521848,173854085302
2022-01-01,ZA16NQJR,ST8000NM0055,8001563222016,0,84,234265456,89,100,8,100,0,95,2891515537,54,40970,100,100,8,100,0,100,0,67,33,100,15,69,62343,33,33,100,0,100,0,200,0,40936,83209739680,302120432268
2022-01-01,ZA130TTW,ST8000DM002,8001563222016,0,83,221445736,96,100,2,100,0,94,2740044896,48,46053,100,100,2,100,0,100,0,70,30,100,0,74,52477,30,30,100,0,100,0,200,0,46007,92979000192,334927052596


The table indicates the models and counts of the hard drive snapshots of hard drives that *never* experienced a failure in the first quarter of 2022.  

In [104]:
# The PySpark DataFrame ('q1_2022_seagate_running_sdf') is grouped by
# 'model' and a count is performed.
# Lastly, a sort is performed on the 'model' column in ascending order.
(
    q1_2022_seagate_running_sdf
    .groupBy('model')
    .count()
    .sort('model')
)

model,count
ST10000NM001G,1027
ST10000NM0086,106608
ST12000NM0007,117492
ST12000NM0008,1811824
ST12000NM001G,1098737
ST12000NM0117,810
ST14000NM001G,965247
ST14000NM0138,143530
ST16000NM001G,1091539
ST4000DM000,1664504


For validation, 5 serial numbers associated with a hard drive failure are checked against the dataframe of hard drive snapshots that are operational and *never* experienced a failure in the first quarter of 2022. The validation is successful as no results are returned.  

In [105]:
# A list ('sample_seagate_sn_fail_list') is created with a sample of
# 5 serial numbers that failed.
sample_seagate_sn_fail_list = ['S300Z5M7', 'S300Z79M', 
                               'S300Z7WQ', 'S300ZAFS', 'S3010LL5']

# The list ('sample_seagate_sn_fail_list') is used to validate that
# the serial numbers do not exist in the PySpark DataFrame
# ('q1_2022_seagate_running_sdf') in the 'serial_number' column.
q1_2022_seagate_running_sdf.filter(F.col('serial_number')
                                   .isin(sample_seagate_sn_fail_list))

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw


A windows function is used to partition the data by serial number and then order by the date in descending order. A row number (last column) is added to the dataframe to track the order.    

In [106]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.partitionBy.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.WindowSpec.partitionBy.html
# https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/expressions/WindowSpec.html

# A PySpark windowSpec is created defining a partition by the
# 'serial_number' column and then order by the 'date' column in
# descending order.
windowSpec  = (
    W.partitionBy('serial_number')
    .orderBy(F.desc('date'))
)

In [107]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.partitionBy.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.WindowSpec.partitionBy.html
# https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/expressions/WindowSpec.html

# The PySpark windowSpec is applied and a 'row_number' column is added 
# to PySpark DataFrame ('q1_2022_seagate_prefail_sdf') to observe the
# functionality.
(
    q1_2022_seagate_prefail_sdf
    .withColumn('row_number', F.row_number()
                .over(windowSpec))
    .limit(10)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw,row_number
2022-01-02,S300Z5M7,ST4000DM000,4000787030016,0,115,84823720,93,100,7,100,0,87,589569260,34,58140,100,100,7,100,0,100,0,76,24,100,0,79,42224,24,24,100,0,100,0,200,0,57973,70379942064,351152087274,1
2022-01-01,S300Z5M7,ST4000DM000,4000787030016,0,114,75358600,93,100,7,100,0,87,587872835,34,58116,100,100,7,100,0,100,0,77,23,100,0,79,42224,23,23,100,0,100,0,200,0,57949,70368379624,350942473802,2
2022-03-01,S300Z79M,ST4000DM000,4000787030016,0,120,242237200,92,100,9,100,0,90,965003010,33,59057,100,100,9,100,0,100,0,77,23,100,0,12,176537,23,23,100,0,100,0,200,0,58239,65768023816,339177725704,1
2022-02-28,S300Z79M,ST4000DM000,4000787030016,0,118,169513616,92,100,9,100,0,90,962894389,33,59033,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58215,65759378320,339024336379,2
2022-02-27,S300Z79M,ST4000DM000,4000787030016,0,118,168894320,92,100,9,100,0,90,961668929,33,59010,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58191,65753340456,338765779115,3
2022-02-26,S300Z79M,ST4000DM000,4000787030016,0,109,23965656,92,100,9,100,0,90,958981212,33,58986,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58167,65747186328,338441439896,4
2022-02-25,S300Z79M,ST4000DM000,4000787030016,0,118,198868056,92,100,9,100,0,90,956964756,33,58962,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58143,65742050600,338064622507,5
2022-02-24,S300Z79M,ST4000DM000,4000787030016,0,115,96035072,92,100,9,100,0,90,955380181,33,58938,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,58120,65739105504,337766275603,6
2022-02-23,S300Z79M,ST4000DM000,4000787030016,0,112,44414104,92,100,9,100,0,90,954101988,33,58914,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,58096,65737225712,337525510773,7
2022-02-22,S300Z79M,ST4000DM000,4000787030016,0,117,161388640,92,100,9,100,0,90,953164899,33,58891,100,100,9,100,0,100,0,80,20,100,0,12,176537,20,20,100,0,100,0,200,0,58072,65735095952,337355021470,8


The windows function is applied to the dataframe to create a new dataframe with the row numbers between 1 and 6 that represent the last 6 days before the hard drive failure. Also, the failure indicator is updated from 0 (`functional`) to 1 (`non-functional`). Lastly, the column with the row number is dropped.    

In [108]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.partitionBy.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.WindowSpec.partitionBy.html
# https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/expressions/WindowSpec.html

# A PySpark DataFrame ('q1_2022_seagate_prefail_recent_6_sdf') is 
# created from the PySpark DataFrame ('q1_2022_seagate_prefail_sdf') 
# where a PySpark windowSpec is applied and a 'row_number' column is 
# added.
# A filter is applied to filter the 'row_number' column between 1 and 
# 6.
# The 'failure' column is cast to '1' representing 'non-functional'.
q1_2022_seagate_prefail_recent_6_sdf = (
    q1_2022_seagate_prefail_sdf
    .withColumn('row_number', F.row_number()
                .over(windowSpec))
    .filter(F.col('row_number').between(1, 6))
    .withColumn('failure', F.lit(1))                         
    .drop('row_number')
)

There are 2,407 hard drive snapshots that capture the 6 days prior to the failure.  

In [109]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_prefail_recent_6_sdf)

(2407, 40)

In [110]:
# The first 10 rows of the PySpark DataFrame 
# ('q1_2022_seagate_prefail_recent_6_sdf') are displayed.
q1_2022_seagate_prefail_recent_6_sdf.limit(12)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-02,S300Z5M7,ST4000DM000,4000787030016,1,115,84823720,93,100,7,100,0,87,589569260,34,58140,100,100,7,100,0,100,0,76,24,100,0,79,42224,24,24,100,0,100,0,200,0,57973,70379942064,351152087274
2022-01-01,S300Z5M7,ST4000DM000,4000787030016,1,114,75358600,93,100,7,100,0,87,587872835,34,58116,100,100,7,100,0,100,0,77,23,100,0,79,42224,23,23,100,0,100,0,200,0,57949,70368379624,350942473802
2022-03-01,S300Z79M,ST4000DM000,4000787030016,1,120,242237200,92,100,9,100,0,90,965003010,33,59057,100,100,9,100,0,100,0,77,23,100,0,12,176537,23,23,100,0,100,0,200,0,58239,65768023816,339177725704
2022-02-28,S300Z79M,ST4000DM000,4000787030016,1,118,169513616,92,100,9,100,0,90,962894389,33,59033,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58215,65759378320,339024336379
2022-02-27,S300Z79M,ST4000DM000,4000787030016,1,118,168894320,92,100,9,100,0,90,961668929,33,59010,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58191,65753340456,338765779115
2022-02-26,S300Z79M,ST4000DM000,4000787030016,1,109,23965656,92,100,9,100,0,90,958981212,33,58986,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58167,65747186328,338441439896
2022-02-25,S300Z79M,ST4000DM000,4000787030016,1,118,198868056,92,100,9,100,0,90,956964756,33,58962,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,58143,65742050600,338064622507
2022-02-24,S300Z79M,ST4000DM000,4000787030016,1,115,96035072,92,100,9,100,0,90,955380181,33,58938,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,58120,65739105504,337766275603
2022-02-10,S300Z7WQ,ST4000DM000,4000787030016,1,119,219087464,92,100,9,100,0,87,523422781,36,56701,100,100,9,68,32,100,0,68,32,100,1,89,22668,32,32,100,16,100,16,200,0,56569,71994834904,980851138574
2022-02-09,S300Z7WQ,ST4000DM000,4000787030016,1,119,221462664,92,100,9,100,0,87,522019632,36,56677,100,100,9,68,32,100,0,68,32,100,1,89,22668,32,32,100,16,100,16,200,0,56545,71981223688,980583512234


The windows function is also applied to the dataframe to create a new dataframe with the row numbers that are **not** between 1 and 6 and do not represent the last 6 days before the hard drive failure. Also, the failure indicator should be 0 (`functional`), but out of caution, the failure indicator is updated to 0 (`functional`). Lastly, the column with the row number is dropped.  

In [111]:
# References
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Window.partitionBy.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.WindowSpec.partitionBy.html
# https://spark.apache.org/docs/latest/api/java/org/apache/spark/sql/expressions/WindowSpec.html

# A PySpark DataFrame ('q1_2022_seagate_prefail_not_6_sdf') is created
# from the PySpark DataFrame ('q1_2022_seagate_prefail_sdf') where a
# PySpark windowSpec is applied and a 'row_number' column is added.
# A filter is applied to filter the 'row_number' column that are *not* 
# between 1 and 6.
# The 'failure' column is cast to '0' representing 'functional'.
# Lastly, the 'row_number' is no longer needed and dropped.
q1_2022_seagate_prefail_not_6_sdf = (
    q1_2022_seagate_prefail_sdf
    .withColumn('row_number', F.row_number()
                .over(windowSpec))
    .filter(~F.col('row_number').between(1, 6))
    .withColumn('failure', F.lit(0))                         
    .drop('row_number')
)

There are 16,564 hard drive snapshots that capture the days prior to the failure with the exception of the last 6 days, which are already represented in the previous dataframe.  

In [112]:
sdf_shape(q1_2022_seagate_prefail_not_6_sdf)

(16564, 40)

In [113]:
# The first 10 rows of the PySpark DataFrame 
# ('q1_2022_seagate_prefail_not_6_sdf') are displayed.
q1_2022_seagate_prefail_not_6_sdf.limit(12)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-02-23,S300Z79M,ST4000DM000,4000787030016,0,112,44414104,92,100,9,100,0,90,954101988,33,58914,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,58096,65737225712,337525510773
2022-02-22,S300Z79M,ST4000DM000,4000787030016,0,117,161388640,92,100,9,100,0,90,953164899,33,58891,100,100,9,100,0,100,0,80,20,100,0,12,176537,20,20,100,0,100,0,200,0,58072,65735095952,337355021470
2022-02-21,S300Z79M,ST4000DM000,4000787030016,0,118,197712152,92,100,9,100,0,90,951213893,33,58867,100,100,9,100,0,100,0,80,20,100,0,12,176537,20,20,100,0,100,0,200,0,58048,65730344568,337214626984
2022-02-20,S300Z79M,ST4000DM000,4000787030016,0,116,117242832,92,100,9,100,0,90,949388377,33,58843,100,100,9,100,0,100,0,80,20,100,0,12,176537,20,20,100,0,100,0,200,0,58025,65727231824,336982208069
2022-02-19,S300Z79M,ST4000DM000,4000787030016,0,114,80186568,92,100,9,100,0,90,947880787,33,58819,100,100,9,100,0,100,0,80,20,100,0,12,176537,20,20,100,0,100,0,200,0,58001,65719820824,336765111058
2022-02-18,S300Z79M,ST4000DM000,4000787030016,0,116,106463240,92,100,9,100,0,90,945659010,33,58796,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,57977,65714305560,336595934476
2022-02-17,S300Z79M,ST4000DM000,4000787030016,0,113,55055472,92,100,9,100,0,90,944397810,33,58772,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,57953,65704511344,336374777201
2022-02-16,S300Z79M,ST4000DM000,4000787030016,0,117,148792064,92,100,9,100,0,90,943642504,33,58760,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,57941,65695658032,336252217858
2022-02-15,S300Z79M,ST4000DM000,4000787030016,0,117,123971040,92,100,9,100,0,90,942255670,33,58736,100,100,9,100,0,100,0,79,21,100,0,12,176537,21,21,100,0,100,0,200,0,57918,65678830144,336012611024
2022-02-14,S300Z79M,ST4000DM000,4000787030016,0,119,219593672,92,100,9,100,0,90,941160082,33,58712,100,100,9,100,0,100,0,78,22,100,0,12,176537,22,22,100,0,100,0,200,0,57894,65665856408,335865125046


In [114]:
# References
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html

# A union is utilized to combine all of the previously created PySpark
# DataFrame into a single PySpark DataFrame 
# (`q1_2022_seagate_sdf_final`).
#
# PySpark DataFrames representing functional (0) snapshots:
# q1_2022_seagate_running_sdf
# q1_2022_seagate_prefail_not_6_sdf
#
# PySpark DataFrames representing non-functional (1) snapshots:
# q1_2022_seagate_fail_sdf
# q1_2022_seagate_prefail_recent_6_sdf
q1_2022_seagate_sdf_final = (
    q1_2022_seagate_running_sdf
    .union(q1_2022_seagate_fail_sdf)
    .union(q1_2022_seagate_prefail_recent_6_sdf)
    .union(q1_2022_seagate_prefail_not_6_sdf)
).sort('date', 'serial_number')

In [115]:
q1_2022_seagate_sdf_final.limit(10)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-01,S3001HBH,ST4000DM000,4000787030016,0,111,41705776,91,100,26,100,0,87,564826045,38,55044,100,100,26,100,0,100,0,73,27,100,3,95,11576,27,27,100,0,100,0,200,0,54939,68556580648,370398316667
2022-01-01,S3003TK3,ST4000DM000,4000787030016,0,118,169130600,97,100,7,100,0,85,390128894,36,56329,100,100,3,100,0,100,0,72,28,100,0,76,49702,28,28,100,0,100,0,200,0,56060,67729811048,409848867140
2022-01-01,S3003W81,ST4000DM000,4000787030016,0,118,189175216,92,100,12,100,0,83,208738421,34,58422,100,100,11,100,0,100,0,83,17,100,2,93,14304,17,17,100,0,100,0,200,0,58377,69088192984,714479920544
2022-01-01,S3004E7J,ST4000DM000,4000787030016,0,109,21882992,92,100,10,100,0,87,621911450,38,54620,100,100,10,100,0,100,0,84,16,100,0,93,15804,16,16,100,0,100,0,200,0,54564,60854732600,355300153266
2022-01-01,S3004L7F,ST4000DM000,4000787030016,0,117,154406968,92,100,11,100,0,82,187608333,38,54620,100,100,11,100,0,100,0,82,18,100,0,94,12487,18,18,100,0,100,0,200,0,54576,55863048616,338697039483
2022-01-01,S30097CV,ST4000DM000,4000787030016,0,116,101982128,91,100,8,100,0,87,555619858,42,51526,100,100,8,100,0,100,0,78,22,100,1,98,4760,22,22,100,0,100,0,200,0,51505,60011178680,429048727801
2022-01-01,S3009TB3,ST4000DM000,4000787030016,0,117,148703288,91,100,16,100,0,84,313950901,39,54309,100,100,16,100,0,100,1,76,24,100,1,98,5376,24,24,100,0,100,0,200,18,54284,66398549120,574796783062
2022-01-01,S300A3S4,ST4000DM000,4000787030016,0,115,90157976,91,100,25,100,0,87,524242074,38,54951,100,100,24,100,0,100,0,72,28,100,0,94,12169,28,28,100,0,100,0,200,0,54867,70510931328,340713293255
2022-01-01,S300ACMY,ST4000DM000,4000787030016,0,118,175219328,92,100,13,100,0,83,218808556,37,55541,100,100,13,100,0,100,0,78,22,100,0,94,12671,22,22,100,0,100,0,200,0,55493,70462480584,363622221514
2022-01-01,S300BBHQ,ST4000DM000,4000787030016,0,120,240713464,92,100,11,100,0,85,339563198,38,54429,100,100,11,100,0,100,0,80,20,100,0,91,19742,20,20,100,0,100,0,200,0,54359,59771758960,427914692240


In [116]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_sdf_final)

(9201120, 40)

Prior to conducting the deterioriating window transformations, there were were 9,200,699 `functional` and 421 `non-functional` hard drive snapshots. After the deterioriating window transformations, there are now 9,198,292 `functinal` and 2,828 `non-functional` hard drive snapshots. By adding the 6 days of hard drive snapshots prior to the failure, the minority class (`non-functional`) has increased by 2,407, more than a factor of 5 increase.  

In [117]:
# The PySpark DataFrame ('q1_2022_seagate_sdf_final') '0' values in the
# 'failure' column are replaced with 'functional' and the '1' values
# are replaced with 'non-functional' using a regular expression
# replace.
# The PySpark DataFrame ('q1_2022_seagate_sdf_final') grouped by
# 'failure' and a count is performed.
(
    q1_2022_seagate_sdf_final
    .withColumn('failure', 
                F.regexp_replace('failure', '0', 'functional'))
    .withColumn('failure', 
                F.regexp_replace('failure', '1', 'non-functional'))
    .groupBy('failure')
    .count()
)

failure,count
functional,9198292
non-functional,2828


The total of 9,201,120 hard drive snapshots has not changed after the transformations, which is expected and suggests that no data was lost.  

In [118]:
q1_2022_seagate_sdf_final.count()

9201120

The table demonstrates the models and the failure status. All models have observations in both classes.  

In [119]:
# The PySpark DataFrame ('q1_2022_seagate_sdf_final') is grouped by 
# 'model', a pivot on 'failure', and a count is # performed.
# The '0' column is renamed to 'functional' and the '1' column is
# renamed to 'non-functional'.
# Lastly, a sort is performed on the 'model' column in ascending 
# order.
(
    q1_2022_seagate_sdf_final
    .groupBy('model')
    .pivot('failure')
    .count()
    .withColumnRenamed('0', 'functional')
    .withColumnRenamed('1', 'non-functional')
    .sort('model')
)

model,functional,non-functional
ST10000NM001G,1027,1
ST10000NM0086,107070,56
ST12000NM0007,118230,116
ST12000NM0008,1814744,472
ST12000NM001G,1099414,128
ST12000NM0117,866,7
ST14000NM001G,965974,152
ST14000NM0138,144216,112
ST16000NM001G,1092200,151
ST4000DM000,1669478,792


To conduct further verification, hard drive snapshots are examined using the `S3010LL5` serial number. Looking at the date in descending order, the 7 most current hard drive snapshots are classified as 1 (`non-functional`), which is expected. The hard drive snapshots beyond the 7 days are classified at 0 (`functional`) which is also expected.  

In [120]:
# The PySpark DataFrame ('q1_2022_seagate_sdf_final') is filtered on 
# 'S3010LL5` from the 'serial_number' column.
# Lastly, a sort is performed on the 'date' column in descending 
# order returning just the 10 first rows.
(
q1_2022_seagate_sdf_final
    .filter(F.col('serial_number') == 'S3010LL5')
    .sort('date', ascending=False)
    .limit(10)
)

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_190_normalized,smart_190_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
2022-01-26,S3010LL5,ST4000DM000,4000787030016,1,105,9911384,92,100,18,100,0,87,480393269,33,58698,100,100,18,1,226,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58479,72509914112,337248390558
2022-01-25,S3010LL5,ST4000DM000,4000787030016,1,109,22121064,92,100,18,100,0,87,479378029,34,58686,100,100,18,1,226,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58467,72500619784,337127547278
2022-01-24,S3010LL5,ST4000DM000,4000787030016,1,117,127098352,92,100,18,100,0,87,478191484,34,58663,100,100,18,1,226,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58444,72489296760,336972108645
2022-01-23,S3010LL5,ST4000DM000,4000787030016,1,106,11680760,92,100,18,100,0,87,476240728,34,58639,100,100,18,1,226,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58420,72466909152,336714569921
2022-01-22,S3010LL5,ST4000DM000,4000787030016,1,117,152406888,92,100,18,100,0,87,474572038,34,58616,100,100,18,1,226,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58397,72450841808,336652609997
2022-01-21,S3010LL5,ST4000DM000,4000787030016,1,117,150114328,92,100,18,100,0,87,472835816,34,58592,100,100,18,1,226,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58373,72435712704,336460788464
2022-01-20,S3010LL5,ST4000DM000,4000787030016,1,118,175402088,92,100,18,100,0,87,470466532,34,58569,100,100,18,1,225,100,0,75,25,100,0,80,40205,25,25,100,160,100,160,200,0,58350,72408322200,336143176491
2022-01-19,S3010LL5,ST4000DM000,4000787030016,0,110,27079392,92,100,18,100,0,86,467605417,34,58534,100,100,18,1,219,100,0,75,25,100,0,80,40203,25,25,100,144,100,144,200,0,58315,72389665248,335631587399
2022-01-18,S3010LL5,ST4000DM000,4000787030016,0,118,200561712,92,100,18,100,0,86,466229144,34,58510,100,100,18,1,214,100,0,76,24,100,0,80,40201,24,24,100,136,100,136,200,0,58291,72377131160,335398664195
2022-01-17,S3010LL5,ST4000DM000,4000787030016,0,109,23017272,92,100,18,100,0,86,464148395,34,58487,100,100,18,1,211,100,0,76,24,100,0,80,40201,24,24,100,136,100,136,200,0,58268,72367774824,335173592122


## Summary

Prior to the data transformation, there were 18,845,260 rows and 179 columns of data.  

In [121]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_sdf)

(18845260, 179)

After the data transformation, there were 9,201,120 rows and 40 columns. A reduction of roughly 51% in rows and nearly a 78% reduction in columns.  

In [122]:
# Display the shape (rows, columns) of the PySpark DataFrame using the
# 'sdf_shape' function.
sdf_shape(q1_2022_seagate_sdf_final)

(9201120, 40)

The data reduction involved:  
* Concentrating on a single manufacturer - SEAGATE  
* Focusing on a specific hard drive type - hard disk drive (HDD)  
* Dropping columns with significant NaN and null (missing values)  
* Dropping rows with more than 40% of data missing  
* Dropping columns with a single (distinct) value  

Lastly, a deteriorating window was produced to classify the last six days of hard drive snapshots prior to failure as `non-functional` (minority class). The effort provides data that might be useful in early classification of a failure, triggering a predictive maintenance action. Also, the effort contributes to improving class imbalance by reclassifying more (over 5 times) `functional` (majority class) to `non-functional` (minority class) in the target.  

In [123]:
# References
# https://sparkbyexamples.com/spark/spark-get-current-number-of-partitions-of-dataframe/

# Display the number of partitions for the
# PySpark dataframe ('q1_2022_seagate_sdf_final').
q1_2022_seagate_sdf_final.rdd.getNumPartitions()

17

## The dataframe is output to a Parquet file

Rather than run the Jupyter Notebook and execute all of the cells, the output of the Notebook can be saved to Parquet file for further analyses.

In [124]:
# References
# https://mungingdata.com/apache-spark/output-one-file-csv-parquet/
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.spark.repartition.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.html

# The PySpark dataframe ('q1_2022_seagate_sdf_final') is repartitioned
# to a single partition ('1') so the output will be a single file.
# The file is written in the Parquet file format to the 
# './data/parquet/q1_2022_seagate.parquet' directory and if the path
# exists, the path and files will be overwritten.
(
    q1_2022_seagate_sdf_final
    .repartition(1)
    .write
    .mode('overwrite')
    .parquet('./data/parquet/q1_2022_seagate.parquet')
)

In [125]:
# References
# https://stackoverflow.com/questions/36905717/un-persisting-all-dataframes-in-pyspark
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Catalog.clearCache.htm

spark.catalog.clearCache()

In [126]:
spark.stop()

## To prevent memory issues with other notebooks, please shutdown the kernel to free up memory or uncomment the cell below and run.  

In [127]:
#quit()

## <center>References</center>   

Apache Spark. (n.d.). <i>Spark configuration.</i>
https://spark.apache.org/docs/latest/configuration.html  
<br>
Beach, B. (2015, February 4). <i>Reliability data set for 41,000 hard drives now open-source.</i> BackBlaze Blog. https://www.backblaze.com/blog/hard-drive-data-feb2015/  
<br>
Klein, A. (2022, March 3). <i>The SSD edition: 2021 drive stats review.</i> Backblaze Blog. https://www.backblaze.com/blog/ssd-edition-2021-drive-stats-review/  
<br>
Pinciroli, R., Yang, L., Alter, J. & Smirni, E. (2022, December 22). <i>The life and death of SSDs and HDDs: Similarities, differences, and prediction models.</i> arXiv. https://arxiv.org/abs/2012.12373v1  
<br>
S.M.A.R.T. (2022, September 9). In <i>Wikipedia</i>. https://en.wikipedia.org/w/index.php?title=S.M.A.R.T.&oldid=1109378579  
<br>
Seagate. (n.d.). <i>My system reported a S.M.A.R.T. error on the drive.</i> Seagate Support. https://www.seagate.com/support/kb/my-system-reported-a-smart-error-on-the-drive-184619en/  
<br>
Shen, J., Ren, Y., Wan, J., & Lan, Y. (2021, February 21). Hard disk drive failure prediction for mobile edge computing based on an LSTM recurrent neural network. <i>Deep Learning in Mobile Computing: Architecture, Applications, and Future Challenges, 21(8878364).</i> https://doi.org/10.1155/2021/8878364  
<br>
Xu, C., Wang, G., Liu, X., Guo, D., & Liu, T. (2016, November). Health status assessment and failure prediction for hard drives with recurrent neural networks. <i>IEEE Transactions On Computers, 65(11).</i> https://chang-xu.github.io/public/disk_prediction.pdf   